# TCGA RACIAL DISPARITY ANALYSIS


## LOAD LIBRARIES AND SET ENVIRONMENTAL VARIABLESS

In [1]:
from IPython.display import display, HTML,IFrame,Image
display(HTML(data="""
<style>
    div#notebook-container    { width: 70%; }
    div#menubar-container     { width: 80%; }
    div#maintoolbar-container { width: 99%; }
</style>
"""))

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

from notebook.services.config import ConfigManager
c = ConfigManager()
c.update('notebook', {"CodeCell": {"cm_config": {"autoCloseBrackets": False}}})
c.update('notebook', {"CodeCell": {"cm_config": {"lineWrapping": True}}})

{'Cell': {'cm_config': {'lineNumbers': True}},
 'CodeCell': {'cm_config': {'autoCloseBrackets': False, 'lineWrapping': True}}}

In [3]:
#Note: First needed to install r-essentials via conda:  /home/mayo/m187735/s212975.Wickland_Immunomics/pvactools/conda/bin/conda install --name=jupyter -c conda-forge r-essential

%env R_HOME=/research/bsi/projects/PI/tertiary/Asmann_Yan_wangy3/s212975.Wickland_Immunomics/pvactools/conda/envs/jupyter/lib/R
#%env LD_LIBRARY_PATH=/research/bsi/projects/PI/tertiary/Asmann_Yan_wangy3/s212975.Wickland_Immunomics/pvactools/conda/envs/jupyter/lib/R/library
#%env PATH=/research/bsi/projects/PI/tertiary/Asmann_Yan_wangy3/s212975.Wickland_Immunomics/pvactools/conda/envs/jupyter/lib/R/lib

%reload_ext rpy2.ipython

env: R_HOME=/research/bsi/projects/PI/tertiary/Asmann_Yan_wangy3/s212975.Wickland_Immunomics/pvactools/conda/envs/jupyter/lib/R


/research/bsi/projects/PI/tertiary/Asmann_Yan_wangy3/s212975.Wickland_Immunomics/pvactools/conda/envs/jupyter/lib/python3.6/site-packages/rpy2/ipython/rmagic.py:77: UserWarning: The Python package `pandas` is strongly recommended when using `rpy2.ipython`. Unfortunately it could not be loaded (error: this version of pandas is incompatible with numpy < 1.15.4
your numpy version is 1.12.1.
Please upgrade numpy to >= 1.15.4 to use this pandas version), but at least we found `numpy`.
  'but at least we found `numpy`.' % str(ie))


In [4]:
%%R
#install packages using jupyter conda environment
library(forcats)
library(ggplot2)
library(scales)
library(reshape2)
library(sjPlot)
library(lemon)
library(ggpubr)
library(repr)
library(tidyr)
library(stringr)
library(plyr)
library(dplyr)
library(ggpubr)
library(shades)
library(ggforce)

R[write to console]: Registered S3 methods overwritten by 'parameters':
  method                           from      
  as.double.parameters_kurtosis    datawizard
  as.double.parameters_skewness    datawizard
  as.double.parameters_smoothness  datawizard
  as.numeric.parameters_kurtosis   datawizard
  as.numeric.parameters_skewness   datawizard
  as.numeric.parameters_smoothness datawizard
  print.parameters_distribution    datawizard
  print.parameters_kurtosis        datawizard
  print.parameters_skewness        datawizard
  summary.parameters_kurtosis      datawizard
  summary.parameters_skewness      datawizard

R[write to console]: Learn more about sjPlot with 'browseVignettes("sjPlot")'.

R[write to console]: 
Attaching package: 'lemon'


R[write to console]: The following objects are masked from 'package:ggplot2':

    CoordCartesian, element_render


R[write to console]: 
Attaching package: 'tidyr'


R[write to console]: The following object is masked from 'package:reshape2':


# PART 1 - CALCULATE TOTAL EXOME READ DEPTH AND CREATE TABLES OF SAMPLE INFORMATION
## Group samples by cancer type, by sequencing center, by tumor purity, by sequencing center, by capture kit

## A. CALCULATE OVERALL EXOME READ DEPTH
• For each TCGA patient for given cancer type  
• For mapped reads, all reads, and unmapped reads  
• Now also for HLA region

### Script to run: [calc_exome_read_depth_in_batches.sh](./Scripts/calc_exome_read_depth_in_batches.sh)

In [ ]:
%%bash

for CANCER_NAME in {BRCA,LUAD,COAD,PRAD,UCEC,KIRC,KIRP};
do
    bash /research/bsi/projects/PI/tertiary/Asmann_Yan_wangy3/s212975.Wickland_Immunomics/GitHub/Racial_Disparities_in_Exome_Read_Depth/Scripts/calc_exome_read_depth_in_batches.sh ${CANCER_NAME} mapped_reads;

    bash /research/bsi/projects/PI/tertiary/Asmann_Yan_wangy3/s212975.Wickland_Immunomics/GitHub/Racial_Disparities_in_Exome_Read_Depth/Scripts/calc_exome_read_depth_in_batches.sh ${CANCER_NAME} all_reads;
  
    bash /research/bsi/projects/PI/tertiary/Asmann_Yan_wangy3/s212975.Wickland_Immunomics/GitHub/Racial_Disparities_in_Exome_Read_Depth/Scripts/calc_exome_read_depth_in_batches.sh ${CANCER_NAME} unmapped_reads;
  
  
done;



### For BRCA, also calculate total read depth *within* Nimblegen capture regions -- note that only samples captured by the three nimblegen kits should be retained (do that later, as right now I'm just doing it on all BRCA samples)

In [ ]:
%%bash

bash /research/bsi/projects/PI/tertiary/Asmann_Yan_wangy3/s212975.Wickland_Immunomics/GitHub/Racial_Disparities_in_Exome_Read_Depth/Scripts/calc_exome_read_depth_in_batches.sh BRCA mapped_reads_within_nimblegen_SeqCap_EZ_Exome_v2_hg38liftover_SORTED

bash /research/bsi/projects/PI/tertiary/Asmann_Yan_wangy3/s212975.Wickland_Immunomics/GitHub/Racial_Disparities_in_Exome_Read_Depth/Scripts/calc_exome_read_depth_in_batches.sh BRCA mapped_reads_within_nimblegen_SeqCap_EZ_Exome_v3_hg19_capture_targets_hg38liftover_SORTED

bash /research/bsi/projects/PI/tertiary/Asmann_Yan_wangy3/s212975.Wickland_Immunomics/GitHub/Racial_Disparities_in_Exome_Read_Depth/Scripts/calc_exome_read_depth_in_batches.sh BRCA mapped_reads_within_nimblegen_hg18_nimblegen_exome_version_2_hg38liftover_SORTED

### Combine results of above script

In [ ]:
%%bash 

CANCER_NAME=BRCA
MAPPED_OR_ALL=mapped_reads_within_nimblegen_SeqCap_EZ_Exome_v2_hg38liftover_SORTED
#mapped_reads_within_nimblegen_SeqCap_EZ_Exome_v2_hg38liftover_SORTED
#mapped_reads_within_nimblegen_SeqCap_EZ_Exome_v3_hg19_capture_targets_hg38liftover_SORTED
#mapped_reads_within_nimblegen_hg18_nimblegen_exome_version_2_hg38liftover_SORTED


#when above qsubs are done, run the following to combine everything:
cat /home/mayo/m187735/s212975.Wickland_Immunomics/processing/TCGA/overall_read_depths/${MAPPED_OR_ALL}/${CANCER_NAME}/read_depths*.txt > /home/mayo/m187735/s212975.Wickland_Immunomics/processing/TCGA/overall_read_depths/${MAPPED_OR_ALL}/${CANCER_NAME}/all_samples_read_depths.txt

#make sure that number of entries in all_read_depths.txt matches TOTAL_NUMBER_OF_BAMS
echo "Number of bams with profiled read depths: "`wc -l /home/mayo/m187735/s212975.Wickland_Immunomics/processing/TCGA/overall_read_depths/${MAPPED_OR_ALL}/${CANCER_NAME}/all_samples_read_depths.txt | awk '{print $1}'`
TOTAL_NUMBER_OF_BAMS=`wc -l /home/mayo/m187735/s212975.Wickland_Immunomics/processing/TCGA/overall_read_depths/${MAPPED_OR_ALL}/${CANCER_NAME}/${CANCER_NAME}_bam_list.txt | cut -d ' ' -f1`
echo "Total number of mforge bams for cancer type: "$TOTAL_NUMBER_OF_BAMS

#if above are not equal, pinpoint which file(s) have no read depth output
# for file in  /home/mayo/m187735/s212975.Wickland_Immunomics/processing/TCGA/overall_read_depths/${MAPPED_OR_ALL}/${CANCER_NAME}/*samples*; do echo `basename $file`;cut -f2 $file | wc -l;done;
 
#manually delete any lines that don't have read depth

#also make sure all analyzed bams are unique
echo "Number of unique samples with profiled read depths: "`cut -f1 /home/mayo/m187735/s212975.Wickland_Immunomics/processing/TCGA/overall_read_depths/${MAPPED_OR_ALL}/${CANCER_NAME}/all_samples_read_depths.txt | sort | uniq | wc -l`

## B. Create master table for each cancer type (updated 11/16/2021)
- Clinical data (from TCGA)  
- Variant data (from protected MAF)  
- Overall exome read depth (from above script)  
- FILTERING STEPS:
    - Only keep samples with both tumor and matched normal exome bam
    - For samples with more than one bam for particular tissue type, keep the one with highest read depth

### Script to run: [master_table_variants_depths_clinical.R](./Scripts/master_table_variants_depths_clinical.R)

In [16]:
%%bash

#note: added one more column to the Immune_Thorsson file (TMB); if downstream problems, revert back to original file
for CANCER_NAME in {BRCA,LUAD,COAD,UCEC,PRAD,KIRC,KIRP}; do 
/research/bsi/projects/PI/tertiary/Asmann_Yan_wangy3/s212975.Wickland_Immunomics/tools/R-3.6.2/bin/Rscript --vanilla /research/bsi/projects/PI/tertiary/Asmann_Yan_wangy3/s212975.Wickland_Immunomics/GitHub/Racial_Disparities_in_Exome_Read_Depth/Scripts/master_table_variants_depths_clinical.R ${CANCER_NAME} mapped_reads exome;done

for CANCER_NAME in {BRCA,LUAD,COAD,UCEC,PRAD,KIRC,KIRP}; do 
/research/bsi/projects/PI/tertiary/Asmann_Yan_wangy3/s212975.Wickland_Immunomics/tools/R-3.6.2/bin/Rscript --vanilla /research/bsi/projects/PI/tertiary/Asmann_Yan_wangy3/s212975.Wickland_Immunomics/GitHub/Racial_Disparities_in_Exome_Read_Depth/Scripts/master_table_variants_depths_clinical.R ${CANCER_NAME} all_reads exome;done

for CANCER_NAME in {BRCA,LUAD,COAD,UCEC,PRAD,KIRC,KIRP}; do 
/research/bsi/projects/PI/tertiary/Asmann_Yan_wangy3/s212975.Wickland_Immunomics/tools/R-3.6.2/bin/Rscript --vanilla /research/bsi/projects/PI/tertiary/Asmann_Yan_wangy3/s212975.Wickland_Immunomics/GitHub/Racial_Disparities_in_Exome_Read_Depth/Scripts/master_table_variants_depths_clinical.R ${CANCER_NAME} unmapped_reads exome;done


CANCER_NAME=BRCA; 
/research/bsi/projects/PI/tertiary/Asmann_Yan_wangy3/s212975.Wickland_Immunomics/tools/R-3.6.2/bin/Rscript --vanilla /research/bsi/projects/PI/tertiary/Asmann_Yan_wangy3/s212975.Wickland_Immunomics/GitHub/Racial_Disparities_in_Exome_Read_Depth/Scripts/master_table_variants_depths_clinical.R ${CANCER_NAME} mapped_reads_within_nimblegen_SeqCap_EZ_Exome_v2_hg38liftover_SORTED exome

CANCER_NAME=BRCA; 
/research/bsi/projects/PI/tertiary/Asmann_Yan_wangy3/s212975.Wickland_Immunomics/tools/R-3.6.2/bin/Rscript --vanilla /research/bsi/projects/PI/tertiary/Asmann_Yan_wangy3/s212975.Wickland_Immunomics/GitHub/Racial_Disparities_in_Exome_Read_Depth/Scripts/master_table_variants_depths_clinical.R ${CANCER_NAME} mapped_reads_within_nimblegen_SeqCap_EZ_Exome_v3_hg19_capture_targets_hg38liftover_SORTED exome

CANCER_NAME=BRCA; 
/research/bsi/projects/PI/tertiary/Asmann_Yan_wangy3/s212975.Wickland_Immunomics/tools/R-3.6.2/bin/Rscript --vanilla /research/bsi/projects/PI/tertiary/Asmann_Yan_wangy3/s212975.Wickland_Immunomics/GitHub/Racial_Disparities_in_Exome_Read_Depth/Scripts/master_table_variants_depths_clinical.R ${CANCER_NAME} mapped_reads_within_nimblegen_hg18_nimblegen_exome_version_2_hg38liftover_SORTED exome



[1] "/research/bsi/projects/PI/tertiary/Asmann_Yan_wangy3/s212975.Wickland_Immunomics/tools/R-3.6.2/lib64/R/library"
[1] "/research/bsi/projects/PI/tertiary/Asmann_Yan_wangy3/s212975.Wickland_Immunomics/tools/R-3.6.2/lib64/R/library"
[1] "/research/bsi/projects/PI/tertiary/Asmann_Yan_wangy3/s212975.Wickland_Immunomics/tools/R-3.6.2/lib64/R/library"
[1] "/research/bsi/projects/PI/tertiary/Asmann_Yan_wangy3/s212975.Wickland_Immunomics/tools/R-3.6.2/lib64/R/library"
[1] "/research/bsi/projects/PI/tertiary/Asmann_Yan_wangy3/s212975.Wickland_Immunomics/tools/R-3.6.2/lib64/R/library"
[1] "/research/bsi/projects/PI/tertiary/Asmann_Yan_wangy3/s212975.Wickland_Immunomics/tools/R-3.6.2/lib64/R/library"
[1] "/research/bsi/projects/PI/tertiary/Asmann_Yan_wangy3/s212975.Wickland_Immunomics/tools/R-3.6.2/lib64/R/library"
[1] "/research/bsi/projects/PI/tertiary/Asmann_Yan_wangy3/s212975.Wickland_Immunomics/tools/R-3.6.2/lib64/R/library"
[1] "/research/bsi/projects/PI/tertiary/Asmann_Yan_wangy3/s21297

# PART 2 - PLOT PRINCIPAL COMPONENTS OF GENOTYPES OF ALL INDIVIDUALS FROM 7 CANCER TYPES AS WELL AS 1000 GENOMES (updated 11/8/21)

## 1. 1000 Genomes Dataset

### Download hg38 VCFs 
- See release notes here: http://ftp.1000genomes.ebi.ac.uk/vol1/ftp/release/20130502/supporting/GRCh38_positions/README_GRCh38_liftover_20170504.txt

In [ ]:
%%bash

mkdir -p /research/bsi/projects/PI/tertiary/Asmann_Yan_wangy3/s212975.Wickland_Immunomics/processing/TCGA/processed_data/ThsndGnms/hg38

cd /research/bsi/projects/PI/tertiary/Asmann_Yan_wangy3/s212975.Wickland_Immunomics/processing/TCGA/processed_data/ThsndGnms/hg38

for i in {1..22}; do wget http://ftp.1000genomes.ebi.ac.uk/vol1/ftp/data_collections/1000_genomes_project/release/20190312_biallelic_SNV_and_INDEL/ALL.chr${i}.shapeit2_integrated_snvindels_v2a_27022019.GRCh38.phased.vcf.gz.tbi;done;


### 2. Merge all chromosomes 

In [ ]:
%%bash

cat <(zcat /research/bsi/projects/PI/tertiary/Asmann_Yan_wangy3/s212975.Wickland_Immunomics/processing/TCGA/processed_data/ThsndGnms/hg38/ALL.chr1.shapeit2_integrated_snvindels_v2a_27022019.GRCh38.phased.vcf.gz | grep -v ^\#\# | head -n1) <(zcat /research/bsi/projects/PI/tertiary/Asmann_Yan_wangy3/s212975.Wickland_Immunomics/processing/TCGA/processed_data/ThsndGnms/hg38/*chr*.vcf.gz | grep -v ^\#) > /research/bsi/projects/PI/tertiary/Asmann_Yan_wangy3/s212975.Wickland_Immunomics/processing/TCGA/processed_data/ThsndGnms/hg38/Thnsd_Gnms_all_chr.vcf

### 2b. Remove the samples for whom we have no population information and who are NOT EUR or AFR or EAS!
- Get psam from here: https://www.dropbox.com/s/a3pfx4w2jlru50b/chrM_phase3_corrected.psam?dl=1
- Make col 3 and col 4 all 0

In [ ]:
%%bash

awk 'NR==FNR{c[$1]++;next};c[$1] >  0' <(head -n1 /research/bsi/projects/PI/tertiary/Asmann_Yan_wangy3/s212975.Wickland_Immunomics/processing/TCGA/processed_data/ThsndGnms/hg38/Thnsd_Gnms_all_chr.vcf | sed 's/\t/\n/g' | sed -n '10,$p') <(grep -e EUR -e AFR -e EAS /research/bsi/projects/PI/tertiary/Asmann_Yan_wangy3/s212975.Wickland_Immunomics/processing/TCGA/processed_data/ThsndGnms/hg38/chrM_phase3_corrected.psam?dl=1 | cut -f1) > /research/bsi/projects/PI/tertiary/Asmann_Yan_wangy3/s212975.Wickland_Immunomics/processing/TCGA/processed_data/ThsndGnms/hg38/list_of_samples_with_info.txt

In [ ]:
%%bash

cat <(head -n1 /research/bsi/projects/PI/tertiary/Asmann_Yan_wangy3/s212975.Wickland_Immunomics/processing/TCGA/processed_data/ThsndGnms/hg38/chrM_phase3_corrected.psam?dl=1) <(awk 'NR==FNR{c[$1]++;next};c[$1] >  0' /research/bsi/projects/PI/tertiary/Asmann_Yan_wangy3/s212975.Wickland_Immunomics/processing/TCGA/processed_data/ThsndGnms/hg38/list_of_samples_with_info.txt /research/bsi/projects/PI/tertiary/Asmann_Yan_wangy3/s212975.Wickland_Immunomics/processing/TCGA/processed_data/ThsndGnms/hg38/chrM_phase3_corrected.psam?dl=1) | sed 's/\#//'> /research/bsi/projects/PI/tertiary/Asmann_Yan_wangy3/s212975.Wickland_Immunomics/processing/TCGA/processed_data/ThsndGnms/hg38/ThnsdGnms_corrected_use.psam 


### 3. Convert to plink format (pgen, pvar, psam)

In [ ]:
%%bash

/home/mayo/m187735/s212975.Wickland_Immunomics/tools/plink2 --vcf /research/bsi/projects/PI/tertiary/Asmann_Yan_wangy3/s212975.Wickland_Immunomics/processing/TCGA/processed_data/ThsndGnms/hg38/Thnsd_Gnms_all_chr.vcf --keep /research/bsi/projects/PI/tertiary/Asmann_Yan_wangy3/s212975.Wickland_Immunomics/processing/TCGA/processed_data/ThsndGnms/hg38/list_of_samples_with_info.txt --make-pgen --out /research/bsi/projects/PI/tertiary/Asmann_Yan_wangy3/s212975.Wickland_Immunomics/processing/TCGA/processed_data/ThsndGnms/hg38/Thnsd_Gnms_all_chr

### 4. Focus on biallelic variants

In [ ]:
%%bash

/home/mayo/m187735/s212975.Wickland_Immunomics/tools/plink2 --pfile /research/bsi/projects/PI/tertiary/Asmann_Yan_wangy3/s212975.Wickland_Immunomics/processing/TCGA/processed_data/ThsndGnms/hg38/Thnsd_Gnms_all_chr --max-alleles 2 --make-bed --chr 1-22 XY --out /research/bsi/projects/PI/tertiary/Asmann_Yan_wangy3/s212975.Wickland_Immunomics/processing/TCGA/processed_data/ThsndGnms/hg38/Thnsd_Gnms_all_chr


### 5. Rename bim file for reference so that each variant named according to chromosomal coordinates

In [ ]:
%%bash

mv /research/bsi/projects/PI/tertiary/Asmann_Yan_wangy3/s212975.Wickland_Immunomics/processing/TCGA/processed_data/ThsndGnms/hg38/Thnsd_Gnms_all_chr.bim /research/bsi/projects/PI/tertiary/Asmann_Yan_wangy3/s212975.Wickland_Immunomics/processing/TCGA/processed_data/ThsndGnms/hg38/Thnsd_Gnms_all_chr.bim.tmp
awk  '{ gsub($2,$1":"$4,$2); print}' OFS="\t" /research/bsi/projects/PI/tertiary/Asmann_Yan_wangy3/s212975.Wickland_Immunomics/processing/TCGA/processed_data/ThsndGnms/hg38/Thnsd_Gnms_all_chr.bim.tmp > /research/bsi/projects/PI/tertiary/Asmann_Yan_wangy3/s212975.Wickland_Immunomics/processing/TCGA/processed_data/ThsndGnms/hg38/Thnsd_Gnms_all_chr.bim

### 6. Create bed file of those coordinates (78 Mb)
- For individual loci, do (chr) (locus - 1) (locus)

In [18]:
%%bash

awk '{print "chr"$1,$4-1,$4}' OFS='\t' /research/bsi/projects/PI/tertiary/Asmann_Yan_wangy3/s212975.Wickland_Immunomics/processing/TCGA/processed_data/ThsndGnms/hg38/Thnsd_Gnms_all_chr.bim > /research/bsi/projects/PI/tertiary/Asmann_Yan_wangy3/s212975.Wickland_Immunomics/processing/TCGA/processed_data/ThsndGnms/hg38/Thnsd_Gnms_biallelic_positions.bed

### 7. Modify PSAM file for easier plotting later

In [26]:
%%R

THSND_GNMS_PSAM <- read.table('/research/bsi/projects/PI/tertiary/Asmann_Yan_wangy3/s212975.Wickland_Immunomics/processing/TCGA/processed_data/ThsndGnms/hg38/ThnsdGnms_corrected_use.psam',header=TRUE)
names(THSND_GNMS_PSAM)[5] <- 'Pop'
THSND_GNMS_PSAM$Pop <- paste0('1000 Genomes: ',THSND_GNMS_PSAM$Pop)


write.table(THSND_GNMS_PSAM,paste0('/research/bsi/projects/PI/tertiary/Asmann_Yan_wangy3/s212975.Wickland_Immunomics/processing/TCGA/processed_data/ThsndGnms/hg38/ThnsdGnms_corrected_use_FOR_PLOT.psam'),sep='\t',row.names=FALSE,quote=FALSE)


## 2. Gather and process data: TCGA


### Create lists of Normal samples and their bam file locations from seven TCGA cancer types
- Note that for any individuals with multiple normal bams, only the bam with greatest overall depth is retained (step in one of the above scripts)
- Autosomes only -- no sex chromosomes


In [2]:
%%bash

CANCER_NAME=BRCA

echo "Total no. normal bams: " `grep Blood /research/bsi/projects/PI/tertiary/Asmann_Yan_wangy3/s212975.Wickland_Immunomics/TCGA_metadata/exome_bams/${CANCER_NAME}_bams_metadata.txt | wc -l`


echo "No. (paired) normals with read depth info: " `cat /research/bsi/projects/PI/tertiary/Asmann_Yan_wangy3/s212975.Wickland_Immunomics/processing/TCGA/processed_data/overall_read_depths_and_sample_information/${CANCER_NAME}_all_reads_exome_master_table.txt | awk '{print $1,$4,$5,$6}' FS='\t' OFS='\t' | awk '$2 == "Blood_Normal" && $3 != 0' FS='\t' | wc -l`

Total no. normal bams:  966
No. (paired) normals with read depth info:  955


In [101]:
%%bash

for CANCER_NAME in BRCA PRAD LUAD UCEC KIRC COAD KIRP; do
    
   echo " " > /research/bsi/projects/PI/tertiary/Asmann_Yan_wangy3/s212975.Wickland_Immunomics/TCGA_metadata/exome_bams/${CANCER_NAME}_normals_list_for_ancestry.txt #must create empty first line; otherwise skips when generating qsubs
    
   cat /research/bsi/projects/PI/tertiary/Asmann_Yan_wangy3/s212975.Wickland_Immunomics/processing/TCGA/processed_data/overall_read_depths_and_sample_information/${CANCER_NAME}_all_reads_exome_master_table.txt | awk '{print $1,$4,$5,$6}' FS='\t' OFS='\t' | awk '$2 == "Blood_Normal" && $3 != 0' FS='\t' | cut -f1,4 >> /research/bsi/projects/PI/tertiary/Asmann_Yan_wangy3/s212975.Wickland_Immunomics/TCGA_metadata/exome_bams/${CANCER_NAME}_normals_list_for_ancestry.txt;
     
done;

### Run HaplotypeCaller on those bams

In [21]:
%%bash

CANCER_NAME=BRCA
BAM_LIST=/research/bsi/projects/PI/tertiary/Asmann_Yan_wangy3/s212975.Wickland_Immunomics/TCGA_metadata/exome_bams/${CANCER_NAME}_normals_list_for_ancestry.txt
BED_FILE=/research/bsi/projects/PI/tertiary/Asmann_Yan_wangy3/s212975.Wickland_Immunomics/processing/TCGA/processed_data/ThsndGnms/hg38/Thnsd_Gnms_biallelic_positions.bed
OUTPUT_DIR=s212975.Wickland_Immunomics
#OUTPUT_DIR=s303213.Marenco_RNAseq_A549
#OUTPUT_DIR=s214917.TCGA_LUAD_AND_LUSC_PROTAC

#create scripts for sets of 2 samples
TOTAL_NUMBER_OF_BAMS=`wc -l ${BAM_LIST} | cut -d ' ' -f1`
NUMBER_OF_SPLITS=`echo $((TOTAL_NUMBER_OF_BAMS/2))`
BAM_LOOP_END=`echo $((NUMBER_OF_SPLITS*2))`

mkdir -p  /home/mayo/m187735/${OUTPUT_DIR}/processing/TCGA/processed_data/HaplotypeCaller_on_TCGA_bams/NORMALS_FOR_ANCESTRY/${CANCER_NAME}
rm -r  /home/mayo/m187735/s303213.Marenco_RNAseq_A549/commands_to_qsub/TCGA/HaplotypeCaller_on_TCGA_bams/NORMALS_FOR_ANCESTRY/${CANCER_NAME}
mkdir -p /home/mayo/m187735/s303213.Marenco_RNAseq_A549/commands_to_qsub/TCGA/HaplotypeCaller_on_TCGA_bams/NORMALS_FOR_ANCESTRY/${CANCER_NAME}

COUNTER=1
COUNTER2=3
#the positional variables in awk correspond to the order of cols in the bam_list.txt generated above
until [ ${COUNTER} -eq $(($BAM_LOOP_END+3)) ];  do awk -v a=${COUNTER} -v b=${COUNTER2} "NR==a,NR==b" ${BAM_LIST} | while read line; do awk -v CANCER_NAME=${CANCER_NAME} -v BED_FILE=${BED_FILE} -v OUTPUT_DIR=${OUTPUT_DIR} '{print "if [ ! -f /home/mayo/m187735/"OUTPUT_DIR"/processing/TCGA/processed_data/HaplotypeCaller_on_TCGA_bams/NORMALS_FOR_ANCESTRY/"CANCER_NAME"/"$1".g.vcf.idx ]; then java -jar /research/bsi/projects/PI/tertiary/Asmann_Yan_wangy3/s212975.Wickland_Immunomics/tools/GenomeAnalysisTK.jar -T HaplotypeCaller -R /research/bsi/projects/PI/tertiary/Asmann_Yan_wangy3/s212975.Wickland_Immunomics/GRCh38.d1.vd1.fa -I "$2" -ERC GVCF --intervals "BED_FILE" -o  /home/mayo/m187735/"OUTPUT_DIR"/processing/TCGA/processed_data/HaplotypeCaller_on_TCGA_bams/NORMALS_FOR_ANCESTRY/"CANCER_NAME"/"$1".g.vcf;fi"}' OFS='\t' > /home/mayo/m187735/s303213.Marenco_RNAseq_A549/commands_to_qsub/TCGA/HaplotypeCaller_on_TCGA_bams/NORMALS_FOR_ANCESTRY/${CANCER_NAME}/${CANCER_NAME}_${COUNTER}-${COUNTER2}.qsub;done;COUNTER=`echo $((${COUNTER}+2))`;COUNTER2=`echo $((${COUNTER}+2))`;done;

In [23]:
%%bash

CANCER_NAME=BRCA

#submit qsubs
for file in /home/mayo/m187735/s303213.Marenco_RNAseq_A549/commands_to_qsub/TCGA/HaplotypeCaller_on_TCGA_bams/NORMALS_FOR_ANCESTRY/${CANCER_NAME}/*qsub; do qsub -l h_vmem=10G -e /home/mayo/m187735/s303213.Marenco_RNAseq_A549/commands_to_qsub/TCGA/HaplotypeCaller_on_TCGA_bams/NORMALS_FOR_ANCESTRY/${CANCER_NAME}/`basename ${file} .qsub`.err -o /home/mayo/m187735/s303213.Marenco_RNAseq_A549/commands_to_qsub/TCGA/HaplotypeCaller_on_TCGA_bams/NORMALS_FOR_ANCESTRY/${CANCER_NAME}/`basename ${file} .qsub`.OUT -pe threaded 4 -q 1-day,4-day,7-day,30-day $file;done;

Your job 1046835 ("BRCA_1-3.qsub") has been submitted
Your job 1046836 ("BRCA_101-103.qsub") has been submitted
Your job 1046837 ("BRCA_103-105.qsub") has been submitted
Your job 1046838 ("BRCA_105-107.qsub") has been submitted
Your job 1046839 ("BRCA_107-109.qsub") has been submitted
Your job 1046840 ("BRCA_109-111.qsub") has been submitted
Your job 1046841 ("BRCA_11-13.qsub") has been submitted
Your job 1046842 ("BRCA_111-113.qsub") has been submitted
Your job 1046843 ("BRCA_113-115.qsub") has been submitted
Your job 1046844 ("BRCA_115-117.qsub") has been submitted
Your job 1046845 ("BRCA_117-119.qsub") has been submitted
Your job 1046846 ("BRCA_119-121.qsub") has been submitted
Your job 1046847 ("BRCA_121-123.qsub") has been submitted
Your job 1046848 ("BRCA_123-125.qsub") has been submitted
Your job 1046849 ("BRCA_125-127.qsub") has been submitted
Your job 1046850 ("BRCA_127-129.qsub") has been submitted
Your job 1046851 ("BRCA_129-131.qsub") has been submitted
Your job 1046852 ("B

In [5]:
%%bash
### LUAD,BRCA STRAGGLES WITH MANUALLY CREATED INDEXES

#echo "java -jar /research/bsi/projects/PI/tertiary/Asmann_Yan_wangy3/s212975.Wickland_Immunomics/tools/GenomeAnalysisTK.jar -T HaplotypeCaller -R /research/bsi/projects/PI/tertiary/Asmann_Yan_wangy3/s212975.Wickland_Immunomics/GRCh38.d1.vd1.fa -I /research/bsi/projects/PI/tertiary/Asmann_Yan_wangy3/s214917.TCGA_LUAD_AND_LUSC_PROTAC/processing/C509.TCGA-55-8208-10A-01D-2238-08.1_gdc_realn.bam -ERC GVCF --intervals /research/bsi/projects/PI/tertiary/Asmann_Yan_wangy3/s212975.Wickland_Immunomics/processing/TCGA/processed_data/ThsndGnms/hg38/Thnsd_Gnms_biallelic_positions.bed -o  /home/mayo/m187735/s214917.TCGA_LUAD_AND_LUSC_PROTAC/processing/TCGA/processed_data/HaplotypeCaller_on_TCGA_bams/NORMALS_FOR_ANCESTRY/LUAD/TCGA-55-8208-10A-01D-2238-08.g.vcf" | qsub -l h_vmem=10G -e /home/mayo/m187735/s303213.Marenco_RNAseq_A549/commands_to_qsub/TCGA/HaplotypeCaller_on_TCGA_bams/NORMALS_FOR_ANCESTRY/LUAD/TCGA-55-8208-10A-01D-2238-08.err -o /home/mayo/m187735/s303213.Marenco_RNAseq_A549/commands_to_qsub/TCGA/HaplotypeCaller_on_TCGA_bams/NORMALS_FOR_ANCESTRY/LUAD/TCGA-55-8208-10A-01D-2238-08.OUT -pe threaded 4 -q 1-day,4-day,7-day,30-day -N TCGA-55-8208-10A-01D-2238-08



#echo "java -jar /research/bsi/projects/PI/tertiary/Asmann_Yan_wangy3/s212975.Wickland_Immunomics/tools/GenomeAnalysisTK.jar -T HaplotypeCaller -R /research/bsi/projects/PI/tertiary/Asmann_Yan_wangy3/s212975.Wickland_Immunomics/GRCh38.d1.vd1.fa -I /research/bsi/projects/PI/tertiary/Asmann_Yan_wangy3/s214917.TCGA_LUAD_AND_LUSC_PROTAC/processing/C509.TCGA-62-A46R-10A-01D-A24F-08.1_gdc_realn.bam -ERC GVCF --intervals /research/bsi/projects/PI/tertiary/Asmann_Yan_wangy3/s212975.Wickland_Immunomics/processing/TCGA/processed_data/ThsndGnms/hg38/Thnsd_Gnms_biallelic_positions.bed -o  /home/mayo/m187735/s214917.TCGA_LUAD_AND_LUSC_PROTAC/processing/TCGA/processed_data/HaplotypeCaller_on_TCGA_bams/NORMALS_FOR_ANCESTRY/LUAD/TCGA-62-A46R-10A-01D-A24F-08.g.vcf" | qsub -l h_vmem=10G -e /home/mayo/m187735/s303213.Marenco_RNAseq_A549/commands_to_qsub/TCGA/HaplotypeCaller_on_TCGA_bams/NORMALS_FOR_ANCESTRY/LUAD/TCGA-62-A46R-10A-01D-A24F-08.err -o /home/mayo/m187735/s303213.Marenco_RNAseq_A549/commands_to_qsub/TCGA/HaplotypeCaller_on_TCGA_bams/NORMALS_FOR_ANCESTRY/LUAD/TCGA-62-A46R-10A-01D-A24F-08.OUT -pe threaded 4 -q 1-day,4-day,7-day,30-day -N LUAD_TCGA-62-A46R-10A-01D-A24F-08



#echo "java -jar /research/bsi/projects/PI/tertiary/Asmann_Yan_wangy3/s212975.Wickland_Immunomics/tools/GenomeAnalysisTK.jar -T HaplotypeCaller -R /research/bsi/projects/PI/tertiary/Asmann_Yan_wangy3/s212975.Wickland_Immunomics/GRCh38.d1.vd1.fa -I /research/bsi/projects/PI/tertiary/Asmann_Yan_wangy3/s214917.TCGA_LUAD_AND_LUSC_PROTAC/processing/C509.TCGA-MP-A4TC-10A-01D-A24P-08.5_gdc_realn.bam -ERC GVCF --intervals /research/bsi/projects/PI/tertiary/Asmann_Yan_wangy3/s212975.Wickland_Immunomics/processing/TCGA/processed_data/ThsndGnms/hg38/Thnsd_Gnms_biallelic_positions.bed -o  /home/mayo/m187735/s214917.TCGA_LUAD_AND_LUSC_PROTAC/processing/TCGA/processed_data/HaplotypeCaller_on_TCGA_bams/NORMALS_FOR_ANCESTRY/LUAD/TCGA-MP-A4TC-10A-01D-A24P-08.g.vcf" | qsub -l h_vmem=10G -e /home/mayo/m187735/s303213.Marenco_RNAseq_A549/commands_to_qsub/TCGA/HaplotypeCaller_on_TCGA_bams/NORMALS_FOR_ANCESTRY/LUAD/TCGA-MP-A4TC-10A-01D-A24P-08.err -o /home/mayo/m187735/s303213.Marenco_RNAseq_A549/commands_to_qsub/TCGA/HaplotypeCaller_on_TCGA_bams/NORMALS_FOR_ANCESTRY/LUAD/TCGA-MP-A4TC-10A-01D-A24P-08.OUT -pe threaded 4 -q 1-day,4-day,7-day,30-day -N LUAD_TCGA-MP-A4TC-10A-01D-A24P-08

echo "java -jar /research/bsi/projects/PI/tertiary/Asmann_Yan_wangy3/s212975.Wickland_Immunomics/tools/GenomeAnalysisTK.jar -T HaplotypeCaller -R /research/bsi/projects/PI/tertiary/Asmann_Yan_wangy3/s212975.Wickland_Immunomics/GRCh38.d1.vd1.fa -I /research/bsi/projects/PI/tertiary/Asmann_Yan_wangy3/s212975.Wickland_Immunomics/processing/scratch/52eb322cfa74f917722c18e6d874a7aa_gdc_realn.bam -ERC GVCF --intervals /research/bsi/projects/PI/tertiary/Asmann_Yan_wangy3/s212975.Wickland_Immunomics/processing/TCGA/processed_data/ThsndGnms/hg38/Thnsd_Gnms_biallelic_positions.bed -o  /home/mayo/m187735/s212975.Wickland_Immunomics/processing/TCGA/processed_data/HaplotypeCaller_on_TCGA_bams/NORMALS_FOR_ANCESTRY/BRCA/TCGA-A2-A1G1-10A-01D-A13O-09.g.vcf" | qsub -l h_vmem=10G -e /home/mayo/m187735/s303213.Marenco_RNAseq_A549/commands_to_qsub/TCGA/HaplotypeCaller_on_TCGA_bams/NORMALS_FOR_ANCESTRY/BRCA/TCGA-A2-A1G1-10A-01D-A13O-09.err -o /home/mayo/m187735/s303213.Marenco_RNAseq_A549/commands_to_qsub/TCGA/HaplotypeCaller_on_TCGA_bams/NORMALS_FOR_ANCESTRY/BRCA/TCGA-A2-A1G1-10A-01D-A13O-09.OUT -pe threaded 4 -q 1-day,4-day,7-day,30-day -N BRCA_TCGA-A2-A1G1-10A-01D-A13O-09


Your job 1059396 ("BRCA_TCGA-A2-A1G1-10A-01D-A13O-09") has been submitted


#### Monitor progress of HaplotypeCaller jobs

In [24]:
%%bash


CANCER_NAME=BRCA


#OUTPUT_DIR=s212975.Wickland_Immunomics
#OUTPUT_DIR=s303213.Marenco_RNAseq_A549
#OUTPUT_DIR=s214917.TCGA_LUAD_AND_LUSC_PROTAC



echo "BAMS COMPLETED: " `ls /home/mayo/m187735/*/processing/TCGA/processed_data/HaplotypeCaller_on_TCGA_bams/NORMALS_FOR_ANCESTRY/${CANCER_NAME}/*idx| wc -l`

echo "BAMS INCOMPLETE: " `awk 'NR==FNR{c[$1]++;next};c[$1] ==  0' <(for file in  /home/mayo/m187735/*/processing/TCGA/processed_data/HaplotypeCaller_on_TCGA_bams/NORMALS_FOR_ANCESTRY/${CANCER_NAME}/*.idx; do basename $file;done | cut -d '.' -f1) <( cat /research/bsi/projects/PI/tertiary/Asmann_Yan_wangy3/s212975.Wickland_Immunomics/TCGA_metadata/exome_bams/${CANCER_NAME}_normals_list_for_ancestry.txt | cut -f1 )`



BAMS COMPLETED:  955
BAMS INCOMPLETE: 


### Run Genotype GVCFs

In [12]:
%%bash

CANCER_NAME=BRCA

#default is nt 6 threaded 7 Xmx49
#if keeps failing, do nt 11 threaded 12, Xmx79 OR nt 22 threaded 20 Xmx24g

#WHICH_INPUT=s212975.Wickland_Immunomics
#WHICH_INPUT=s303213.Marenco_RNAseq_A549
#WHICH_INPUT=s214917.TCGA_LUAD_AND_LUSC_PROTAC


INPUT_DIR=/home/mayo/m187735/*/processing/TCGA/processed_data/HaplotypeCaller_on_TCGA_bams/NORMALS_FOR_ANCESTRY/${CANCER_NAME}
OUTPUT_DIR=/research/bsi/projects/PI/tertiary/Asmann_Yan_wangy3/s212975.Wickland_Immunomics/processing/TCGA/processed_data/GenotypeGVCFs_on_TCGA/NORMALS_FOR_ANCESTRY/${CANCER_NAME}

mkdir -p ${OUTPUT_DIR}/logs

for GVCF in ${INPUT_DIR}/*g.vcf; do GVCFS_LIST="${GVCFS_LIST} --variant ${GVCF}"; done

for CHR in 17 ;do

    

    echo "java -Xmx49g -jar /research/bsi/projects/PI/tertiary/Asmann_Yan_wangy3/s212975.Wickland_Immunomics/tools/GenomeAnalysisTK.jar -T GenotypeGVCFs -R /research/bsi/projects/PI/tertiary/Asmann_Yan_wangy3/s212975.Wickland_Immunomics/GRCh38.d1.vd1.fa ${GVCFS_LIST} -o ${OUTPUT_DIR}/${CANCER_NAME}_chr${CHR}2.vcf  --intervals chr${CHR} --disable_auto_index_creation_and_locking_when_reading_rods -nt 18" | qsub -l h_vmem=50G -e ${OUTPUT_DIR}/logs/${CANCER_NAME}_chr${CHR}2_GenotypeGVCFs.err -o ${OUTPUT_DIR}/logs/${CANCER_NAME}_chr${CHR}2_GenotypeGVCFs.out -q 4-day,1-day,7-day,30-day,lg-mem -pe threaded 16 -N ${CANCER_NAME}_chr${CHR}2_GenotypeGVCFs;done
    

Your job 1108853 ("BRCA_chr172_GenotypeGVCFs") has been submitted


### Combine VCFs within cancers

In [14]:
%%bash

CANCER_NAME=BRCA
OUTPUT_DIR=/research/bsi/projects/PI/tertiary/Asmann_Yan_wangy3/s212975.Wickland_Immunomics/processing/TCGA/processed_data/GenotypeGVCFs_on_TCGA/NORMALS_FOR_ANCESTRY/${CANCER_NAME}

for VCF in `ls -v ${OUTPUT_DIR}/*vcf`; 

        do
           VCFS_LIST="${VCFS_LIST} --variant ${VCF}"  ; 
         done;

    java -cp /research/bsi/projects/PI/tertiary/Asmann_Yan_wangy3/s212975.Wickland_Immunomics/tools/GenomeAnalysisTK.jar org.broadinstitute.gatk.tools.CatVariants -R /research/bsi/projects/PI/tertiary/Asmann_Yan_wangy3/s212975.Wickland_Immunomics/GRCh38.d1.vd1.fa ${VCFS_LIST} -out `dirname ${OUTPUT_DIR}`/${CANCER_NAME}.vcf

.........10.........20..
There were no warn messages.


INFO  17:04:54,920 HelpFormatter - ------------------------------------------------------- 
INFO  17:04:54,922 HelpFormatter - Program Name: org.broadinstitute.gatk.tools.CatVariants 
INFO  17:04:54,925 HelpFormatter - Program Args: -R /research/bsi/projects/PI/tertiary/Asmann_Yan_wangy3/s212975.Wickland_Immunomics/GRCh38.d1.vd1.fa --variant /research/bsi/projects/PI/tertiary/Asmann_Yan_wangy3/s212975.Wickland_Immunomics/processing/TCGA/processed_data/GenotypeGVCFs_on_TCGA/NORMALS_FOR_ANCESTRY/UCEC/UCEC_chr1.vcf --variant /research/bsi/projects/PI/tertiary/Asmann_Yan_wangy3/s212975.Wickland_Immunomics/processing/TCGA/processed_data/GenotypeGVCFs_on_TCGA/NORMALS_FOR_ANCESTRY/UCEC/UCEC_chr2.vcf --variant /research/bsi/projects/PI/tertiary/Asmann_Yan_wangy3/s212975.Wickland_Immunomics/processing/TCGA/processed_data/GenotypeGVCFs_on_TCGA/NORMALS_FOR_ANCESTRY/UCEC/UCEC_chr3.vcf --variant /research/bsi/projects/PI/tertiary/Asmann_Yan_wangy3/s212975.Wickland_Immunomics/processing/TCGA/process

### RUN VQSR

In [36]:
%%bash

#set VQSR variables
REF=/research/bsi/projects/PI/tertiary/Asmann_Yan_wangy3/s212975.Wickland_Immunomics/GRCh38.d1.vd1.fa 
HAPMAP=/research/bsi/projects/PI/tertiary/Asmann_Yan_wangy3/s212975.Wickland_Immunomics/tools/VQSR/hapmap_3.3.hg38.vcf.gz
OMNI=/research/bsi/projects/PI/tertiary/Asmann_Yan_wangy3/s212975.Wickland_Immunomics/tools/VQSR/1000G_omni2.5.hg38.vcf.gz
G1000=/research/bsi/projects/PI/tertiary/Asmann_Yan_wangy3/s212975.Wickland_Immunomics/tools/VQSR/1000G_phase1.snps.high_confidence.hg38.vcf.gz
DBSNP=/research/bsi/projects/PI/tertiary/Asmann_Yan_wangy3/s212975.Wickland_Immunomics/tools/VQSR/Homo_sapiens_assembly38.dbsnp138.vcf
MILLS=/research/bsi/projects/PI/tertiary/Asmann_Yan_wangy3/s212975.Wickland_Immunomics/tools/VQSR/Mills_and_1000G_gold_standard.indels.hg38.vcf.gz


for CANCER in BRCA; do

        rm -r /research/bsi/projects/PI/tertiary/Asmann_Yan_wangy3/s212975.Wickland_Immunomics/processing/TCGA/processed_data/VQSR_on_TCGA/NORMALS_FOR_ANCESTRY/${CANCER}/logs
        mkdir -p /research/bsi/projects/PI/tertiary/Asmann_Yan_wangy3/s212975.Wickland_Immunomics/processing/TCGA/processed_data/VQSR_on_TCGA/NORMALS_FOR_ANCESTRY/${CANCER}/logs



    INPUT=/research/bsi/projects/PI/tertiary/Asmann_Yan_wangy3/s212975.Wickland_Immunomics/processing/TCGA/processed_data/GenotypeGVCFs_on_TCGA/NORMALS_FOR_ANCESTRY/${CANCER}/${CANCER}.vcf
    OUTPUT_DIR=/research/bsi/projects/PI/tertiary/Asmann_Yan_wangy3/s212975.Wickland_Immunomics/processing/TCGA/processed_data/VQSR_on_TCGA/NORMALS_FOR_ANCESTRY/${CANCER}

    mkdir -p ${OUTPUT_DIR}/logs

    ############################
    #SNPs############ NOTE LACK OF INBREEDING COEFFICIENT; THAT DAT JUST NOT PRESENT
    ############################
        echo "java -Xmx10g -jar /research/bsi/projects/PI/tertiary/Asmann_Yan_wangy3/s212975.Wickland_Immunomics/tools/GenomeAnalysisTK.jar -T VariantRecalibrator -R ${REF} -input ${INPUT} -tranche 100.0 -tranche 99.9 -tranche 99.0 -tranche 90.0 -mode SNP -an QD -an MQ -an MQRankSum -an ReadPosRankSum -an FS -an SOR -resource:hapmap,VCF,known=false,training=true,truth=true,prior=15.0 ${HAPMAP} -resource:omni,known=false,training=true,truth=true,prior=12.0 ${OMNI} -resource:1000G,known=false,training=true,truth=false,prior=10.0 ${G1000} -resource:dbsnp,known=true,training=false,truth=false,prior=2.0 ${DBSNP} -recalFile ${OUTPUT_DIR}/${CANCER}_VQSR_recalibrate_SNPs -tranchesFile ${OUTPUT_DIR}/${CANCER}_VQSR_tranches_SNPs -rscriptFile ${OUTPUT_DIR}/${CANCER}_VQSR_SNPs_only.plots.R --disable_auto_index_creation_and_locking_when_reading_rods --target_titv 3.2

        java -Xmx10g -jar /research/bsi/projects/PI/tertiary/Asmann_Yan_wangy3/s212975.Wickland_Immunomics/tools/GenomeAnalysisTK.jar -T ApplyRecalibration -R ${REF} -input ${INPUT} -o ${OUTPUT_DIR}/${CANCER}_VQSR_SNPs_only.vcf  -nt 5 -recalFile ${OUTPUT_DIR}/${CANCER}_VQSR_recalibrate_SNPs -tranchesFile ${OUTPUT_DIR}/${CANCER}_VQSR_tranches_SNPs --ts_filter_level 99.5 -mode SNP" > ${OUTPUT_DIR}/logs/${CANCER}_VQSR.qsub

        ############################
        #INDELs############
        ############################
        echo "java -Xmx10g -jar /research/bsi/projects/PI/tertiary/Asmann_Yan_wangy3/s212975.Wickland_Immunomics/tools/GenomeAnalysisTK.jar -T VariantRecalibrator -R ${REF} -input ${OUTPUT_DIR}/${CANCER}_VQSR_SNPs_only.vcf -tranche 100.0 -tranche 99.9 -tranche 99.0 -tranche 90.0 -mode INDEL -an QD -an MQRankSum -an ReadPosRankSum -an FS -an SOR --maxGaussians 4 -resource:mills,known=false,training=true,truth=true,prior=12.0 ${MILLS} -resource:dbsnp,known=true,training=false,truth=false,prior=2.0 ${DBSNP} -recalFile ${OUTPUT_DIR}/${CANCER}_VQSR_SNPs_recalibrate_INDELs -tranchesFile ${OUTPUT_DIR}/${CANCER}_VQSR_SNPs_tranches_INDELs --disable_auto_index_creation_and_locking_when_reading_rods		

    java -Xmx10g -jar /research/bsi/projects/PI/tertiary/Asmann_Yan_wangy3/s212975.Wickland_Immunomics/tools/GenomeAnalysisTK.jar -T ApplyRecalibration -R ${REF} -input ${OUTPUT_DIR}/${CANCER}_VQSR_SNPs_only.vcf -o ${OUTPUT_DIR}/${CANCER}_VQSR_final.vcf -recalFile ${OUTPUT_DIR}/${CANCER}_VQSR_SNPs_recalibrate_INDELs -tranchesFile ${OUTPUT_DIR}/${CANCER}_VQSR_SNPs_tranches_INDELs --ts_filter_level 99 -mode INDEL" >> ${OUTPUT_DIR}/logs/${CANCER}_VQSR.qsub

        #submit job	
        qsub -l h_vmem=20G -e ${OUTPUT_DIR}/logs/${CANCER}.err -o ${OUTPUT_DIR}/logs/${CANCER}.out -q 1-day -pe threaded 6 ${OUTPUT_DIR}/logs/${CANCER}_VQSR.qsub

done

Your job 1116535 ("BRCA_VQSR.qsub") has been submitted


### Select the PASS variants and modify sample names

In [37]:
%%bash

for CANCER_NAME in BRCA;do

    FILE=/research/bsi/projects/PI/tertiary/Asmann_Yan_wangy3/s212975.Wickland_Immunomics/processing/TCGA/processed_data/VQSR_on_TCGA/NORMALS_FOR_ANCESTRY/${CANCER_NAME}/${CANCER_NAME}_VQSR_final.vcf

    echo "awk '/#/ || /PASS/' ${FILE} | sed 's/H_LS/TCGA/g' | sed 's/H_LK/TCGA/g' | sed 's/H_LR/TCGA/g' | sed 's/Pooled_DNA-2011-03/TCGA-AQ-A04L/' > `dirname ${FILE}`/`basename ${FILE} .vcf`_PASS.vcf" |  qsub -l h_vmem=4G -e /research/bsi/projects/PI/tertiary/Asmann_Yan_wangy3/s212975.Wickland_Immunomics/processing/TCGA/processed_data/VQSR_on_TCGA/NORMALS_FOR_ANCESTRY/${CANCER_NAME}/logs/${CANCER_NAME}_PASS.err -o /research/bsi/projects/PI/tertiary/Asmann_Yan_wangy3/s212975.Wickland_Immunomics/processing/TCGA/processed_data/VQSR_on_TCGA/NORMALS_FOR_ANCESTRY/${CANCER_NAME}/logs/${CANCER_NAME}_PASS.out -q 1-day,lg-mem -pe threaded 1 -N ${CANCER_NAME}_PASS;
    
done;

Your job 1116928 ("BRCA_PASS") has been submitted


### Remove HLA, LCT, 8p and 17 q regions; Remove multiallelic positions
- HLA: chr6:28,477,797-33,448,354 ((https://www.ncbi.nlm.nih.gov/grc/human/regions/MHC?asm=GRCh38))
- LCT: chr2:136,545,420-136,594,754 ((https://www.ncbi.nlm.nih.gov/genome/gdv/browser/gene/?id=3938 // https://www.ncbi.nlm.nih.gov/gene?Cmd=DetailsSearch&Term=3938))
- 8p: 0-45600000 ((https://hgdownload.cse.ucsc.edu/goldenPath/hg38/database/cytoBand.txt.gz))
- 17q: 24000000-81195210 ((https://hgdownload.cse.ucsc.edu/goldenPath/hg38/database/cytoBand.txt.gz))
- Biallelic alleles only

In [4]:
%%bash

for CANCER_NAME in BRCA;do

    FILE=/research/bsi/projects/PI/tertiary/Asmann_Yan_wangy3/s212975.Wickland_Immunomics/processing/TCGA/processed_data/VQSR_on_TCGA/NORMALS_FOR_ANCESTRY/${CANCER_NAME}/${CANCER_NAME}_VQSR_final_PASS.vcf

echo "/home/mayo/m187735/s212975.Wickland_Immunomics/tools/plink2 --vcf ${FILE} --exclude range /research/bsi/projects/PI/tertiary/Asmann_Yan_wangy3/s212975.Wickland_Immunomics/tools/VQSR/HLA_LCT_8p_17q_for_TCGA_racial_hg38.bed --make-bed --max-alleles 2 --out `dirname ${FILE}`/`basename ${FILE} .vcf`_biallelic_without_HLA_LCT_8p_17q --const-fid" | qsub -l h_vmem=10G -e /research/bsi/projects/PI/tertiary/Asmann_Yan_wangy3/s212975.Wickland_Immunomics/processing/TCGA/processed_data/VQSR_on_TCGA/NORMALS_FOR_ANCESTRY/${CANCER_NAME}/logs/${CANCER_NAME}_PASS_biallelic_without_HLA_LCT_8p_17q.err -o /research/bsi/projects/PI/tertiary/Asmann_Yan_wangy3/s212975.Wickland_Immunomics/processing/TCGA/processed_data/VQSR_on_TCGA/NORMALS_FOR_ANCESTRY/${CANCER_NAME}/logs/${CANCER_NAME}_PASS_biallelic_without_HLA_LCT_8p_17q.out -q 1-day,lg-mem -pe threaded 4 -N ${CANCER_NAME}_filter1;
done

Your job 1119054 ("BRCA_filter1") has been submitted


### Remove variants with LD > 0.20 in 0.1 MB windows

In [5]:
%%bash

for CANCER_NAME in BRCA;do

    FILE=/research/bsi/projects/PI/tertiary/Asmann_Yan_wangy3/s212975.Wickland_Immunomics/processing/TCGA/processed_data/VQSR_on_TCGA/NORMALS_FOR_ANCESTRY/${CANCER_NAME}/${CANCER_NAME}_VQSR_final_PASS_biallelic_without_HLA_LCT_8p_17q
    
    echo "/home/mayo/m187735/s212975.Wickland_Immunomics/tools/plink1.9/plink  --bfile ${FILE} --recode vcf --out ${FILE}; /home/mayo/m187735/s212975.Wickland_Immunomics/tools/bcftools-1.10.2/bin/bcftools +prune ${FILE}.vcf  -l 0.20 -w 100kb -Ov -o ${FILE}_LDlessThan02_Window100kb.vcf" | qsub -l h_vmem=10G -e /research/bsi/projects/PI/tertiary/Asmann_Yan_wangy3/s212975.Wickland_Immunomics/processing/TCGA/processed_data/VQSR_on_TCGA/NORMALS_FOR_ANCESTRY/${CANCER_NAME}/logs/${CANCER_NAME}_PASS_biallelic_without_HLA_LCT_8p_17q_LDlessThan02_Window100kb.err -o /research/bsi/projects/PI/tertiary/Asmann_Yan_wangy3/s212975.Wickland_Immunomics/processing/TCGA/processed_data/VQSR_on_TCGA/NORMALS_FOR_ANCESTRY/${CANCER_NAME}/logs/${CANCER_NAME}_PASS_biallelic_without_HLA_LCT_8p_17q_LDlessThan02_Window100kb.out -q 1-day,lg-mem -pe threaded 4 -N ${CANCER_NAME}_filter2;
    done

Your job 1119073 ("BRCA_filter2") has been submitted


### Remove variants with HWE < 0.0005 & MAF

In [6]:
%%bash

for CANCER_NAME in BRCA;do

    FILE=/research/bsi/projects/PI/tertiary/Asmann_Yan_wangy3/s212975.Wickland_Immunomics/processing/TCGA/processed_data/VQSR_on_TCGA/NORMALS_FOR_ANCESTRY/${CANCER_NAME}/${CANCER_NAME}_VQSR_final_PASS_biallelic_without_HLA_LCT_8p_17q_LDlessThan02_Window100kb.vcf

    echo "/home/mayo/m187735/s212975.Wickland_Immunomics/tools/plink2 --vcf ${FILE} --keep-allele-order  --export vcf --maf .05 --hwe .00001 --out `dirname ${FILE}`/`basename ${FILE} .vcf`_HWE00001_MAF05 --const-fid" |  qsub -l h_vmem=10G -e /research/bsi/projects/PI/tertiary/Asmann_Yan_wangy3/s212975.Wickland_Immunomics/processing/TCGA/processed_data/VQSR_on_TCGA/NORMALS_FOR_ANCESTRY/${CANCER_NAME}/logs/${CANCER_NAME}_PASS_biallelic_without_HLA_LCT_8p_17q_LDlessThan02_Window100kb_HWE00001_MAF05.err -o /research/bsi/projects/PI/tertiary/Asmann_Yan_wangy3/s212975.Wickland_Immunomics/processing/TCGA/processed_data/VQSR_on_TCGA/NORMALS_FOR_ANCESTRY/${CANCER_NAME}/logs/${CANCER_NAME}_PASS_biallelic_without_HLA_LCT_8p_17q_LDlessThan02_Window100kb_HWE00001_MAF05.out -q 1-day,lg-mem -pe threaded 4 -N ${CANCER_NAME}_filter3;
    done;



Your job 1119392 ("BRCA_filter3") has been submitted


In [7]:
%%bash
### make ped files from above

for CANCER_NAME in BRCA; do
    VCF=/research/bsi/projects/PI/tertiary/Asmann_Yan_wangy3/s212975.Wickland_Immunomics/processing/TCGA/processed_data/VQSR_on_TCGA/NORMALS_FOR_ANCESTRY/${CANCER_NAME}/${CANCER_NAME}_VQSR_final_PASS_biallelic_without_HLA_LCT_8p_17q_LDlessThan02_Window100kb_HWE00001_MAF05.vcf

    /home/mayo/m187735/s212975.Wickland_Immunomics/tools/plink2 --vcf ${VCF} --make-bed  --out `dirname ${VCF}`/`basename ${VCF} .vcf` --const-fid;
done

PLINK v2.00a3LM 64-bit Intel (11 Oct 2021)     www.cog-genomics.org/plink/2.0/
(C) 2005-2021 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to /research/bsi/projects/PI/tertiary/Asmann_Yan_wangy3/s212975.Wickland_Immunomics/processing/TCGA/processed_data/VQSR_on_TCGA/NORMALS_FOR_ANCESTRY/BRCA/BRCA_VQSR_final_PASS_biallelic_without_HLA_LCT_8p_17q_LDlessThan02_Window100kb_HWE00001_MAF05.log.
Options in effect:
  --const-fid
  --make-bed
  --out /research/bsi/projects/PI/tertiary/Asmann_Yan_wangy3/s212975.Wickland_Immunomics/processing/TCGA/processed_data/VQSR_on_TCGA/NORMALS_FOR_ANCESTRY/BRCA/BRCA_VQSR_final_PASS_biallelic_without_HLA_LCT_8p_17q_LDlessThan02_Window100kb_HWE00001_MAF05
  --vcf /research/bsi/projects/PI/tertiary/Asmann_Yan_wangy3/s212975.Wickland_Immunomics/processing/TCGA/processed_data/VQSR_on_TCGA/NORMALS_FOR_ANCESTRY/BRCA/BRCA_VQSR_final_PASS_biallelic_without_HLA_LCT_8p_17q_LDlessThan02_Window100kb_HWE00001_MAF05.vcf

Start time: Thu Nov 18 0

### Rename bim file for TCGA so that each variant named according to chromosomal coordinates

In [8]:
%%bash

for CANCER_NAME in BRCA; do

    FILE=/research/bsi/projects/PI/tertiary/Asmann_Yan_wangy3/s212975.Wickland_Immunomics/processing/TCGA/processed_data/VQSR_on_TCGA/NORMALS_FOR_ANCESTRY/${CANCER_NAME}/${CANCER_NAME}_VQSR_final_PASS_biallelic_without_HLA_LCT_8p_17q_LDlessThan02_Window100kb_HWE00001_MAF05.bim

    mv ${FILE} ${FILE}.tmp

    awk  '{ gsub($2,$1":"$4,$2); print}' OFS="\t" ${FILE}.tmp > ${FILE}


    rm ${FILE}.tmp;
done;

###  Create psam file for TCGA files; sample information (i.e. race)

In [11]:
%%R


for (CANCER_NAME in c('COAD','UCEC','KIRP','KIRC','PRAD','LUAD','BRCA')){
    
    system(paste0("mkdir -p /research/bsi/projects/PI/tertiary/Asmann_Yan_wangy3/s212975.Wickland_Immunomics/processing/TCGA/processed_data/ThsndGnms/hg38/ANCESTRY_ANALYSIS_FOR_TCGA/",CANCER_NAME))

    SAMPLE_INFO <- read.table(paste0('/research/bsi/projects/PI/tertiary/Asmann_Yan_wangy3/s212975.Wickland_Immunomics/processing/TCGA/processed_data/overall_read_depths_and_sample_information/',CANCER_NAME,'_all_reads_exome_master_table.txt'),header=TRUE,sep='\t')[,c('submitter_id_long','race')]
    SAMPLE_INFO$PAT=0
    SAMPLE_INFO$MAT=0
    SAMPLE_INFO$SEX=1
    names(SAMPLE_INFO)[1:2] <- c('IID','Pop')

    SAMPLE_INFO$Pop <- paste0('TCGA: ',SAMPLE_INFO$Pop)

    write.table(x=SAMPLE_INFO,file=paste0('/research/bsi/projects/PI/tertiary/Asmann_Yan_wangy3/s212975.Wickland_Immunomics/processing/TCGA/processed_data/ThsndGnms/hg38/ANCESTRY_ANALYSIS_FOR_TCGA/',CANCER_NAME,'/',CANCER_NAME,'_sample_info.txt'),sep='\t',row.names=FALSE,quote=FALSE)

}


## 3. Joint PCA analysis of 1000 Genomes and TCGA

### 1. Identify shared/overlapping variants between the two datasets#

In [12]:
%%bash

for CANCER_NAME in BRCA; do


STUDY_FILE=/research/bsi/projects/PI/tertiary/Asmann_Yan_wangy3/s212975.Wickland_Immunomics/processing/TCGA/processed_data/VQSR_on_TCGA/NORMALS_FOR_ANCESTRY/${CANCER_NAME}/${CANCER_NAME}_VQSR_final_PASS_biallelic_without_HLA_LCT_8p_17q_LDlessThan02_Window100kb_HWE00001_MAF05

awk 'NR==FNR{c[$1$2$4$5$6]++;next};c[$1$2$4$5$6] >  0' ${STUDY_FILE}.bim /research/bsi/projects/PI/tertiary/Asmann_Yan_wangy3/s212975.Wickland_Immunomics/processing/TCGA/processed_data/ThsndGnms/hg38/Thnsd_Gnms_all_chr.bim | cut -f2 > /research/bsi/projects/PI/tertiary/Asmann_Yan_wangy3/s212975.Wickland_Immunomics/processing/TCGA/processed_data/ThsndGnms/hg38/ANCESTRY_ANALYSIS_FOR_TCGA/${CANCER_NAME}/shared_SNPs_TCGA_${CANCER_NAME}_1000G.txt ;
done



### 2. Extract shared/overlapping variants from both reference (1000 Genomes) and study (TCGA) dataset

In [13]:
%%bash

for CANCER_NAME in BRCA; do

    SHARED_POSITIONS=/research/bsi/projects/PI/tertiary/Asmann_Yan_wangy3/s212975.Wickland_Immunomics/processing/TCGA/processed_data/ThsndGnms/hg38/ANCESTRY_ANALYSIS_FOR_TCGA/${CANCER_NAME}/shared_SNPs_TCGA_${CANCER_NAME}_1000G.txt

    REF_BFILE=/research/bsi/projects/PI/tertiary/Asmann_Yan_wangy3/s212975.Wickland_Immunomics/processing/TCGA/processed_data/ThsndGnms/hg38/Thnsd_Gnms_all_chr

    STUDY_BFILE=/research/bsi/projects/PI/tertiary/Asmann_Yan_wangy3/s212975.Wickland_Immunomics/processing/TCGA/processed_data/VQSR_on_TCGA/NORMALS_FOR_ANCESTRY/${CANCER_NAME}/${CANCER_NAME}_VQSR_final_PASS_biallelic_without_HLA_LCT_8p_17q_LDlessThan02_Window100kb_HWE00001_MAF05

    #Reference (1000 Genomes)
    /home/mayo/m187735/s212975.Wickland_Immunomics/tools/plink2 --bfile ${REF_BFILE} --keep-allele-order --extract ${SHARED_POSITIONS} --make-bed --out /research/bsi/projects/PI/tertiary/Asmann_Yan_wangy3/s212975.Wickland_Immunomics/processing/TCGA/processed_data/ThsndGnms/hg38/ANCESTRY_ANALYSIS_FOR_TCGA/${CANCER_NAME}/1000G_overlap_with_TCGA_${CANCER_NAME}

    #Study (TCGA)
    /home/mayo/m187735/s212975.Wickland_Immunomics/tools/plink2 --bfile ${STUDY_BFILE} --keep-allele-order --extract ${SHARED_POSITIONS}  --make-bed --out /research/bsi/projects/PI/tertiary/Asmann_Yan_wangy3/s212975.Wickland_Immunomics/processing/TCGA/processed_data/ThsndGnms/hg38/ANCESTRY_ANALYSIS_FOR_TCGA/${CANCER_NAME}/TCGA_${CANCER_NAME}_overlap_with_1000G;
done

PLINK v2.00a3LM 64-bit Intel (11 Oct 2021)     www.cog-genomics.org/plink/2.0/
(C) 2005-2021 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to /research/bsi/projects/PI/tertiary/Asmann_Yan_wangy3/s212975.Wickland_Immunomics/processing/TCGA/processed_data/ThsndGnms/hg38/ANCESTRY_ANALYSIS_FOR_TCGA/BRCA/1000G_overlap_with_TCGA_BRCA.log.
Options in effect:
  --bfile /research/bsi/projects/PI/tertiary/Asmann_Yan_wangy3/s212975.Wickland_Immunomics/processing/TCGA/processed_data/ThsndGnms/hg38/Thnsd_Gnms_all_chr
  --extract /research/bsi/projects/PI/tertiary/Asmann_Yan_wangy3/s212975.Wickland_Immunomics/processing/TCGA/processed_data/ThsndGnms/hg38/ANCESTRY_ANALYSIS_FOR_TCGA/BRCA/shared_SNPs_TCGA_BRCA_1000G.txt
  --keep-allele-order
  --make-bed
  --out /research/bsi/projects/PI/tertiary/Asmann_Yan_wangy3/s212975.Wickland_Immunomics/processing/TCGA/processed_data/ThsndGnms/hg38/ANCESTRY_ANALYSIS_FOR_TCGA/BRCA/1000G_overlap_with_TCGA_BRCA

Start time: Thu Nov 18 07:13

### Attempt merge; will likely fail, but this step needed to get list of 'missnps'

In [14]:
%%bash

for CANCER_NAME in BRCA; do

    STUDY_BFILE=/research/bsi/projects/PI/tertiary/Asmann_Yan_wangy3/s212975.Wickland_Immunomics/processing/TCGA/processed_data/ThsndGnms/hg38/ANCESTRY_ANALYSIS_FOR_TCGA/${CANCER_NAME}/TCGA_${CANCER_NAME}_overlap_with_1000G
    REF_BFILE=/research/bsi/projects/PI/tertiary/Asmann_Yan_wangy3/s212975.Wickland_Immunomics/processing/TCGA/processed_data/ThsndGnms/hg38/ANCESTRY_ANALYSIS_FOR_TCGA/${CANCER_NAME}/1000G_overlap_with_TCGA_${CANCER_NAME}
    OUT_FILE=/research/bsi/projects/PI/tertiary/Asmann_Yan_wangy3/s212975.Wickland_Immunomics/processing/TCGA/processed_data/ThsndGnms/hg38/ANCESTRY_ANALYSIS_FOR_TCGA/${CANCER_NAME}/combined_1000G_TCGA_${CANCER_NAME}

    #Attempt merge
    /home/mayo/m187735/s212975.Wickland_Immunomics/tools/plink1.9/plink --bfile ${STUDY_BFILE} --bmerge ${REF_BFILE} --make-bed --out ${OUT_FILE};
done

    #Merge likely to fail due to multiallelic sites, but still needed to run this to get list of "missnps" to remove from each dataset

PLINK v1.90b6.21 32-bit (19 Oct 2020)          www.cog-genomics.org/plink/1.9/
(C) 2005-2020 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to /research/bsi/projects/PI/tertiary/Asmann_Yan_wangy3/s212975.Wickland_Immunomics/processing/TCGA/processed_data/ThsndGnms/hg38/ANCESTRY_ANALYSIS_FOR_TCGA/BRCA/combined_1000G_TCGA_BRCA.log.
Options in effect:
  --bfile /research/bsi/projects/PI/tertiary/Asmann_Yan_wangy3/s212975.Wickland_Immunomics/processing/TCGA/processed_data/ThsndGnms/hg38/ANCESTRY_ANALYSIS_FOR_TCGA/BRCA/TCGA_BRCA_overlap_with_1000G
  --bmerge /research/bsi/projects/PI/tertiary/Asmann_Yan_wangy3/s212975.Wickland_Immunomics/processing/TCGA/processed_data/ThsndGnms/hg38/ANCESTRY_ANALYSIS_FOR_TCGA/BRCA/1000G_overlap_with_TCGA_BRCA
  --make-bed
  --out /research/bsi/projects/PI/tertiary/Asmann_Yan_wangy3/s212975.Wickland_Immunomics/processing/TCGA/processed_data/ThsndGnms/hg38/ANCESTRY_ANALYSIS_FOR_TCGA/BRCA/combined_1000G_TCGA_BRCA

773957 MB RAM detect

Error: 69 variants with 3+ alleles present.
* If you believe this is due to strand inconsistency, try --flip with
  /research/bsi/projects/PI/tertiary/Asmann_Yan_wangy3/s212975.Wickland_Immunomics/processing/TCGA/processed_data/ThsndGnms/hg38/ANCESTRY_ANALYSIS_FOR_TCGA/BRCA/combined_1000G_TCGA_BRCA-merge.missnp.
  (Warning: if this seems to work, strand errors involving SNPs with A/T or C/G
  alleles probably remain in your data.  If LD between nearby SNPs is high,
  --flip-scan should detect them.)
* If you are dealing with genuine multiallelic variants, we recommend exporting
  that subset of the data to VCF (via e.g. '--recode vcf'), merging with
  another tool/script, and then importing the result; PLINK is not yet suited
  to handling them.
See https://www.cog-genomics.org/plink/1.9/data#merge3 for more discussion.


CalledProcessError: Command 'b'\nfor CANCER_NAME in BRCA; do\n\n    STUDY_BFILE=/research/bsi/projects/PI/tertiary/Asmann_Yan_wangy3/s212975.Wickland_Immunomics/processing/TCGA/processed_data/ThsndGnms/hg38/ANCESTRY_ANALYSIS_FOR_TCGA/${CANCER_NAME}/TCGA_${CANCER_NAME}_overlap_with_1000G\n    REF_BFILE=/research/bsi/projects/PI/tertiary/Asmann_Yan_wangy3/s212975.Wickland_Immunomics/processing/TCGA/processed_data/ThsndGnms/hg38/ANCESTRY_ANALYSIS_FOR_TCGA/${CANCER_NAME}/1000G_overlap_with_TCGA_${CANCER_NAME}\n    OUT_FILE=/research/bsi/projects/PI/tertiary/Asmann_Yan_wangy3/s212975.Wickland_Immunomics/processing/TCGA/processed_data/ThsndGnms/hg38/ANCESTRY_ANALYSIS_FOR_TCGA/${CANCER_NAME}/combined_1000G_TCGA_${CANCER_NAME}\n\n    #Attempt merge\n    /home/mayo/m187735/s212975.Wickland_Immunomics/tools/plink1.9/plink --bfile ${STUDY_BFILE} --bmerge ${REF_BFILE} --make-bed --out ${OUT_FILE};\ndone\n\n    #Merge likely to fail due to multiallelic sites, but still needed to run this to get lis

###  Remove missnps from dataset (i.e. loci with too many variants)

In [15]:
%%bash


for CANCER_NAME in BRCA; do

    STUDY_BFILE=/research/bsi/projects/PI/tertiary/Asmann_Yan_wangy3/s212975.Wickland_Immunomics/processing/TCGA/processed_data/ThsndGnms/hg38/ANCESTRY_ANALYSIS_FOR_TCGA/${CANCER_NAME}/TCGA_${CANCER_NAME}_overlap_with_1000G
    REF_BFILE=/research/bsi/projects/PI/tertiary/Asmann_Yan_wangy3/s212975.Wickland_Immunomics/processing/TCGA/processed_data/ThsndGnms/hg38/ANCESTRY_ANALYSIS_FOR_TCGA/${CANCER_NAME}/1000G_overlap_with_TCGA_${CANCER_NAME}
   
    MISSNPS=/research/bsi/projects/PI/tertiary/Asmann_Yan_wangy3/s212975.Wickland_Immunomics/processing/TCGA/processed_data/ThsndGnms/hg38/ANCESTRY_ANALYSIS_FOR_TCGA/${CANCER_NAME}/combined_1000G_TCGA_${CANCER_NAME}
   
    STUDY_OUT_FILE=/research/bsi/projects/PI/tertiary/Asmann_Yan_wangy3/s212975.Wickland_Immunomics/processing/TCGA/processed_data/ThsndGnms/hg38/ANCESTRY_ANALYSIS_FOR_TCGA/${CANCER_NAME}/TCGA_${CANCER_NAME}_overlap_with_1000G_fixed
    REF_OUT_FILE=/research/bsi/projects/PI/tertiary/Asmann_Yan_wangy3/s212975.Wickland_Immunomics/processing/TCGA/processed_data/ThsndGnms/hg38/ANCESTRY_ANALYSIS_FOR_TCGA/${CANCER_NAME}/1000G_overlap_with_TCGA_${CANCER_NAME}_fixed


    #reference
    /home/mayo/m187735/s212975.Wickland_Immunomics/tools/plink1.9/plink --bfile ${REF_BFILE} --keep-allele-order --exclude ${MISSNPS}-merge.missnp  --make-bed  --out ${REF_OUT_FILE}
    
    #study
    /home/mayo/m187735/s212975.Wickland_Immunomics/tools/plink1.9/plink --bfile ${STUDY_BFILE}  --keep-allele-order --exclude ${MISSNPS}-merge.missnp --make-bed --out ${STUDY_OUT_FILE};
    
done

PLINK v1.90b6.21 32-bit (19 Oct 2020)          www.cog-genomics.org/plink/1.9/
(C) 2005-2020 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to /research/bsi/projects/PI/tertiary/Asmann_Yan_wangy3/s212975.Wickland_Immunomics/processing/TCGA/processed_data/ThsndGnms/hg38/ANCESTRY_ANALYSIS_FOR_TCGA/BRCA/1000G_overlap_with_TCGA_BRCA_fixed.log.
Options in effect:
  --bfile /research/bsi/projects/PI/tertiary/Asmann_Yan_wangy3/s212975.Wickland_Immunomics/processing/TCGA/processed_data/ThsndGnms/hg38/ANCESTRY_ANALYSIS_FOR_TCGA/BRCA/1000G_overlap_with_TCGA_BRCA
  --exclude /research/bsi/projects/PI/tertiary/Asmann_Yan_wangy3/s212975.Wickland_Immunomics/processing/TCGA/processed_data/ThsndGnms/hg38/ANCESTRY_ANALYSIS_FOR_TCGA/BRCA/combined_1000G_TCGA_BRCA-merge.missnp
  --keep-allele-order
  --make-bed
  --out /research/bsi/projects/PI/tertiary/Asmann_Yan_wangy3/s212975.Wickland_Immunomics/processing/TCGA/processed_data/ThsndGnms/hg38/ANCESTRY_ANALYSIS_FOR_TCGA/BRCA/1000

### Re-attempt merge

In [16]:
%%bash

for CANCER_NAME in BRCA; do

    STUDY_BFILE=/research/bsi/projects/PI/tertiary/Asmann_Yan_wangy3/s212975.Wickland_Immunomics/processing/TCGA/processed_data/ThsndGnms/hg38/ANCESTRY_ANALYSIS_FOR_TCGA/${CANCER_NAME}/TCGA_${CANCER_NAME}_overlap_with_1000G_fixed
    REF_BFILE=/research/bsi/projects/PI/tertiary/Asmann_Yan_wangy3/s212975.Wickland_Immunomics/processing/TCGA/processed_data/ThsndGnms/hg38/ANCESTRY_ANALYSIS_FOR_TCGA/${CANCER_NAME}/1000G_overlap_with_TCGA_${CANCER_NAME}_fixed
    OUT_FILE=/research/bsi/projects/PI/tertiary/Asmann_Yan_wangy3/s212975.Wickland_Immunomics/processing/TCGA/processed_data/ThsndGnms/hg38/ANCESTRY_ANALYSIS_FOR_TCGA/${CANCER_NAME}/combined_1000G_TCGA_${CANCER_NAME}_fixed
    
    #Re-attempt merge
    /home/mayo/m187735/s212975.Wickland_Immunomics/tools/plink1.9/plink --bfile ${STUDY_BFILE} --bmerge ${REF_BFILE} --make-bed --out ${OUT_FILE};
    
done;

#Rename fam file for easier plotting
#cp /research/bsi/projects/PI/tertiary/Asmann_Yan_wangy3/s212975.Wickland_Immunomics/processing/ADSP/pruned_SNPs_for_substructure_analysis/Ancestry/ADSP_overlap_with_1000Genomes_fixed.fam /research/bsi/projects/PI/tertiary/Asmann_Yan_wangy3/s212975.Wickland_Immunomics/processing/ADSP/pruned_SNPs_for_substructure_analysis/Ancestry/ADSP.fam

PLINK v1.90b6.21 32-bit (19 Oct 2020)          www.cog-genomics.org/plink/1.9/
(C) 2005-2020 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to /research/bsi/projects/PI/tertiary/Asmann_Yan_wangy3/s212975.Wickland_Immunomics/processing/TCGA/processed_data/ThsndGnms/hg38/ANCESTRY_ANALYSIS_FOR_TCGA/BRCA/combined_1000G_TCGA_BRCA_fixed.log.
Options in effect:
  --bfile /research/bsi/projects/PI/tertiary/Asmann_Yan_wangy3/s212975.Wickland_Immunomics/processing/TCGA/processed_data/ThsndGnms/hg38/ANCESTRY_ANALYSIS_FOR_TCGA/BRCA/TCGA_BRCA_overlap_with_1000G_fixed
  --bmerge /research/bsi/projects/PI/tertiary/Asmann_Yan_wangy3/s212975.Wickland_Immunomics/processing/TCGA/processed_data/ThsndGnms/hg38/ANCESTRY_ANALYSIS_FOR_TCGA/BRCA/1000G_overlap_with_TCGA_BRCA_fixed
  --make-bed
  --out /research/bsi/projects/PI/tertiary/Asmann_Yan_wangy3/s212975.Wickland_Immunomics/processing/TCGA/processed_data/ThsndGnms/hg38/ANCESTRY_ANALYSIS_FOR_TCGA/BRCA/combined_1000G_TCGA_BRCA_fix

### PCA

In [17]:
%%bash

for CANCER_NAME in BRCA; do

    BFILE=/research/bsi/projects/PI/tertiary/Asmann_Yan_wangy3/s212975.Wickland_Immunomics/processing/TCGA/processed_data/ThsndGnms/hg38/ANCESTRY_ANALYSIS_FOR_TCGA/${CANCER_NAME}/combined_1000G_TCGA_${CANCER_NAME}_fixed
    
    /home/mayo/m187735/s212975.Wickland_Immunomics/tools/plink1.9/plink --bfile ${BFILE} --pca --out ${BFILE};
    
done;



PLINK v1.90b6.21 32-bit (19 Oct 2020)          www.cog-genomics.org/plink/1.9/
(C) 2005-2020 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to /research/bsi/projects/PI/tertiary/Asmann_Yan_wangy3/s212975.Wickland_Immunomics/processing/TCGA/processed_data/ThsndGnms/hg38/ANCESTRY_ANALYSIS_FOR_TCGA/BRCA/combined_1000G_TCGA_BRCA_fixed.log.
Options in effect:
  --bfile /research/bsi/projects/PI/tertiary/Asmann_Yan_wangy3/s212975.Wickland_Immunomics/processing/TCGA/processed_data/ThsndGnms/hg38/ANCESTRY_ANALYSIS_FOR_TCGA/BRCA/combined_1000G_TCGA_BRCA_fixed
  --out /research/bsi/projects/PI/tertiary/Asmann_Yan_wangy3/s212975.Wickland_Immunomics/processing/TCGA/processed_data/ThsndGnms/hg38/ANCESTRY_ANALYSIS_FOR_TCGA/BRCA/combined_1000G_TCGA_BRCA_fixed
  --pca

773957 MB RAM detected; reserving 2047 MB for main workspace.
17751 variants loaded from .bim file.
2621 people (0 males, 0 females, 2621 ambiguous) loaded from .fam.
Ambiguous sex IDs written to
/research/bsi/

## 4. PLOT PCA-classified ancestry and add to master tables

In [4]:
%%R

TCGA_samples_all <- data.frame()
TCGA_samples_summary_all <- data.frame()
data_all_cancers <- data.frame()

for (CANCER_NAME in c('BRCA','UCEC','PRAD','LUAD','COAD','KIRP','KIRC')){
  
TCGA_FAM=paste0('/research/bsi/projects/PI/tertiary/Asmann_Yan_wangy3/s212975.Wickland_Immunomics/processing/TCGA/processed_data/ThsndGnms/hg38/ANCESTRY_ANALYSIS_FOR_TCGA/',CANCER_NAME,'/TCGA_',CANCER_NAME,'_overlap_with_1000G_fixed.fam')
EIGENVEC=paste0('/research/bsi/projects/PI/tertiary/Asmann_Yan_wangy3/s212975.Wickland_Immunomics/processing/TCGA/processed_data/ThsndGnms/hg38/ANCESTRY_ANALYSIS_FOR_TCGA/',CANCER_NAME,'/combined_1000G_TCGA_',CANCER_NAME,'_fixed.eigenvec') # the eigenvec file
refSamplesFile=paste0('/research/bsi/projects/PI/tertiary/Asmann_Yan_wangy3/s212975.Wickland_Immunomics/processing/TCGA/processed_data/ThsndGnms/hg38/ThnsdGnms_corrected_use_FOR_PLOT.psam')
refSamplesPop='Pop'
refSamplesIID = 'IID'
studySamplesFile=paste0('/research/bsi/projects/PI/tertiary/Asmann_Yan_wangy3/s212975.Wickland_Immunomics/processing/TCGA/processed_data/ThsndGnms/hg38/ANCESTRY_ANALYSIS_FOR_TCGA/',CANCER_NAME,'/',CANCER_NAME,'_sample_info.txt')
  
refColorsFile=paste0('/research/bsi/projects/PI/tertiary/Asmann_Yan_wangy3/s212975.Wickland_Immunomics/processing/TCGA/processed_data/ThsndGnms/hg38/Colors_Populations.txt')
studyColorsFile=paste0('/research/bsi/projects/PI/tertiary/Asmann_Yan_wangy3/s212975.Wickland_Immunomics/processing/TCGA/processed_data/ThsndGnms/hg38/Colors_TCGA.txt')
  
  
  
  samples <- read.table(TCGA_FAM,header=FALSE)[, 1:2]
  samples$V2 <- gsub("0_TCGA","TCGA",samples$V2)  
  colnames(samples) <- c("FID", "IID")
  
  pca_data <- read.table(EIGENVEC,header=FALSE, stringsAsFactors = FALSE)
  colnames(pca_data) <- c("FID", "IID", paste("PC", 1:(ncol(pca_data) - 2), sep = ""))
  pca_data$IID <- gsub("0_TCGA","TCGA",pca_data$IID)  #DW
  #some end in -1 for some reason; get rid of that!
  pca_data$IID <- gsub("-1$","",pca_data$IID)
  
  refSamples <- read.table(refSamplesFile, header = TRUE, stringsAsFactors = FALSE,sep='\t') #DW
  names(refSamples)[names(refSamples) == refSamplesIID] <- "IID"
  names(refSamples)[names(refSamples) == refSamplesPop] <- "Pop"
  refSamples <- dplyr::select_(refSamples, ~IID, ~Pop)
  refSamples$IID <- as.character(refSamples$IID)
  refSamples$Pop <- as.character(refSamples$Pop)
  studySamples <- read.table(studySamplesFile,header=TRUE,sep='\t')[,c('IID','Pop')] #DW
  
  refColors <- read.table(refColorsFile, header = TRUE, stringsAsFactors = FALSE)
  studyColors <- read.table(studyColorsFile,header=TRUE) #DW
  
  refSamples <- merge(refSamples, refColors, by = "Pop", all.X = TRUE)
  studySamples <- merge(studySamples,studyColors, by = 'Pop',all.X=TRUE) #DW
  allSamples <- rbind(refSamples,studySamples) #DW
  
  data_all <- merge(pca_data, allSamples, by = "IID", all.x = TRUE) #DW
  data_all$Color <- as.factor(data_all$Color)
  
  
  
  
  ##direction of values is irrelevant, so fine to reverse sign
  if (CANCER_NAME == 'UCEC' | CANCER_NAME == 'KIRP'){data_all$PC1 <- data_all$PC1*-1}
  if (CANCER_NAME == 'BRCA'){data_all$PC2 <- data_all$PC2*-1}
  
  data_all$CANCER <- CANCER_NAME
  
  
  
  data_all$Pop <- factor(data_all$Pop, levels=c('1000 Genomes: EAS','1000 Genomes: EUR','1000 Genomes: AFR','  ','',' ','TCGA: Asian','TCGA: White','TCGA: Black','TCGA: American Indian / Alaska Native','TCGA: Native Hawaiian / Pacific Islander','TCGA: Not reported'),ordered=TRUE)
  

  
  #number of samples misclassified by TCGA
  TCGA_samples <- data_all[grepl('TCGA',data_all$IID),][,c('IID','PC1','PC2','Pop','Color')]
  TCGA_samples$PCA_ANCESTRY <- ifelse(TCGA_samples$PC2 < -0.01 & TCGA_samples$PC1 < 0,'EUR',ifelse(TCGA_samples$PC2 > -0.015 & TCGA_samples$PC1 > 0, 'AFR',NA))
  
  TCGA_samples <- TCGA_samples[,c('IID','Pop','PCA_ANCESTRY')]
  names(TCGA_samples) <- c('submitter_id','race_self_reported','ancestry_PCA') 
  TCGA_samples$race_self_reported <- sub('TCGA: ','',TCGA_samples$race_self_reported)
  
  TCGA_samples_before_filter <- TCGA_samples
  TCGA_samples <- droplevels(subset(TCGA_samples, ancestry_PCA=='AFR' | ancestry_PCA =='EUR')) #rm non-AFR/EUR
  
    TCGA_samples_all <- rbind(TCGA_samples_all,TCGA_samples)
  
  #create summary table
  TCGA_samples_summary <- data.frame(Cancer=CANCER_NAME)
  TCGA_samples_summary$`Self-reported White` <- nrow(subset(TCGA_samples_before_filter,race_self_reported == 'White'))
  TCGA_samples_summary$`Self-reported Black` <- nrow(subset(TCGA_samples_before_filter,race_self_reported == 'Black'))
  TCGA_samples_summary$`PCA-classified European` <- nrow(subset(TCGA_samples,ancestry_PCA == 'EUR'))
  TCGA_samples_summary$`PCA-classified African` <-  nrow(subset(TCGA_samples,ancestry_PCA == 'AFR'))
    
TCGA_samples_summary_all <- rbind(TCGA_samples_summary_all, TCGA_samples_summary)

data_all_cancers <- rbind(data_all_cancers,data_all)

data_all_cancers$Pop <- sub('/ ','/\n',data_all_cancers$Pop)

data_all_cancers$Pop <- factor(data_all_cancers$Pop, levels=c('1000 Genomes: EAS','1000 Genomes: EUR','1000 Genomes: AFR','  ','',' ','TCGA: Asian','TCGA: White','TCGA: Black','TCGA: American Indian /\nAlaska Native','TCGA: Native Hawaiian /\nPacific Islander','TCGA: Not reported'),ordered=TRUE)

}





TCGA_samples_summary_all[is.na(TCGA_samples_summary_all)] <- 0 
names(TCGA_samples_summary_all) <- gsub(' ','\n',names(TCGA_samples_summary_all))




#create different layer for each population so that smaller can be superimposed on larger

data_all_cancers$CANCER <- factor(data_all_cancers$CANCER, levels=c('BRCA','UCEC','PRAD','LUAD','COAD','KIRP','KIRC'),ordered=TRUE)

layer_1 <- subset(data_all_cancers, Pop == '1000 Genomes: EAS')
layer_2 <- subset(data_all_cancers, Pop == '1000 Genomes: EUR')
layer_3 <- subset(data_all_cancers, Pop == '1000 Genomes: AFR')
layer_4 <- subset(data_all_cancers, Pop == '')
layer_5 <- subset(data_all_cancers, Pop == ' ')
layer_55 <- subset(data_all_cancers, Pop == ' ')

layer_6 <- subset(data_all_cancers, Pop == 'TCGA: Asian')
layer_7 <- subset(data_all_cancers, Pop == 'TCGA: White')
layer_8 <- subset(data_all_cancers, Pop == 'TCGA: Black')
layer_9 <- subset(data_all_cancers, Pop == 'TCGA: American Indian /\nAlaska Native')
layer_95 <- subset(data_all_cancers, Pop == 'TCGA: Native Hawaiian /\nPacific Islander')
layer_10 <- subset(data_all_cancers, Pop == 'TCGA: Not reported')



PLOT <- ggplot()  +  geom_point(data = layer_1, aes_string(x = "PC1", y = "PC2", color = "Pop"),size=1) +
  geom_point(data = layer_2, aes_string(x = "PC1", y = "PC2", color = "Pop"),size=1.5) +     
  geom_point(data = layer_3, aes_string(x = "PC1", y = "PC2", color = "Pop"),size=1.5) +
  geom_point(data = layer_4, aes_string(x = "PC1", y = "PC2", color = "Pop"),size=1.5) +
  geom_point(data = layer_5, aes_string(x = "PC1", y = "PC2", color = "Pop"),size=1.5) +
  geom_point(data = layer_55, aes_string(x = "PC1", y = "PC2", color = "Pop"),size=1.5) +
  geom_point(data = layer_6, aes_string(x = "PC1", y = "PC2", color = "Pop"),size=2) +
  geom_point(data = layer_8, aes_string(x = "PC1", y = "PC2", color = "Pop"),size=2) +
  geom_point(data = layer_7, aes_string(x = "PC1", y = "PC2", color = "Pop"),size=2) +
  
  geom_point(data = layer_9, aes_string(x = "PC1", y = "PC2", color = "Pop"),size=2) +
  geom_point(data = layer_95, aes_string(x = "PC1", y = "PC2", color = "Pop"),size=2) +
  geom_point(data = layer_10, aes_string(x = "PC1", y = "PC2", color = "Pop"),size=2) +
  
  facet_wrap(~CANCER,nrow=2) + 
  
  
  
  scale_color_manual(values = c(refColors$Color,studyColors$Color),   drop=FALSE) + 
  guides(color = guide_legend(nrow = 2, byrow = TRUE)) +  
  ggtitle('Ancestry of TCGA samples using 1000 Genomes as reference') + theme_bw() + theme(legend.position = "bottom") +
  
  
  guides(colour = guide_legend(override.aes = list(size = 11), 
                               nrow = 6)) + 
  theme(legend.text = element_text(size = 17,margin = margin(t=6,b=6, r=10, unit = "pt")), #incr spacing b/w legnd items
        legend.title = element_blank(), 
        axis.title.y = element_text(size = 18, face = "bold"), 
        axis.title.x = element_text(size = 18, face = "bold",margin = margin(t = 10, r = 0, b = 10, l = 0)), 
        axis.text.x = element_text(size = 14),
        panel.spacing = unit(1.25, "lines"), #spacing b/w facets
        axis.text.y = element_text(size = 14),
        strip.background = element_blank(),
        panel.background = element_rect(color = 'black',fill=NA),
        panel.grid.major = element_blank(),
        panel.grid.minor = element_blank(),
        strip.text.x = element_text(size=20,face='plain'),
        plot.title = element_text(size=22,hjust=0.5,face='bold',margin=margin(b = 15)),
        plot.margin=unit(c(0.3,0.3,0.3,0.3),"cm")) 

LEGEND <- g_legend(PLOT)

dummy2 <- data.frame(X = c("LUAD", "COAD"), Z = c(0.1, 0))

PLOT <- PLOT  +  geom_segment(data=dummy2,aes(y=-0.03,yend=0.01, x=0,xend=0.00),size=1) +
  geom_segment(data=dummy2,aes(y=-.01,yend=-.01, x=-.02,xend=0.03),size=1) + 
theme(legend.position = 'none') 


TABLE <- ggtexttable(as.matrix(TCGA_samples_summary_all), theme= ttheme(
  base_style = "default",
  base_size = 18,
  base_colour = "black",
  padding = unit(c(0.50, 0.45), "cm"),
  colnames.style = colnames_style(size = 16,fill=c('grey70','grey70','grey70','grey70','grey70','grey70','white')),
  rownames.style = rownames_style(size = 16,face = 'plain',hjust=1), #add x= if want to change position
  tbody.style = tbody_style(size = 18)))



png('/research/bsi/projects/PI/tertiary/Asmann_Yan_wangy3/s212975.Wickland_Immunomics/GitHub/Racial_Disparities_in_Exome_Read_Depth/MS_plots/FIGS0.png',height=11,width=15,res=500,units='in')
ggarrange(PLOT,NULL,ggarrange(LEGEND,TABLE,nrow=1),nrow=3,heights=c(0.8,0.10,0.2)) + theme(plot.margin = margin(b=2,l=0.25,t=0.1,r=1.25, "cm")) #incr marrgin at bottom
dev.off()

write.table(x=TCGA_samples_all,file='/research/bsi/projects/PI/tertiary/Asmann_Yan_wangy3/s212975.Wickland_Immunomics/processing/TCGA/processed_data/overall_read_depths_and_sample_information/PCA-classified_ancestry.txt',sep='\t',quote=FALSE,row.names=FALSE)



### Add PCA-classified ancestry to master tables, retaining only the samples classified as EUR or AFR

In [19]:
%%R

PCA_ANCESTRY <- read.table('/research/bsi/projects/PI/tertiary/Asmann_Yan_wangy3/s212975.Wickland_Immunomics/processing/TCGA/processed_data/overall_read_depths_and_sample_information/PCA-classified_ancestry.txt',header=TRUE,sep='\t')[,c('submitter_id','ancestry_PCA')]
PCA_ANCESTRY$submitter_id <- gsub("^([^-]*-[^-]*-[^-]*).*", "\\1",PCA_ANCESTRY$submitter_id) 
PCA_ANCESTRY$race_PCA <- ifelse(PCA_ANCESTRY$ancestry_PCA == 'EUR','White',ifelse(PCA_ANCESTRY$ancestry_PCA == 'AFR','Black',NA))

for (FILE in list.files(paste0('/research/bsi/projects/PI/tertiary/Asmann_Yan_wangy3/s212975.Wickland_Immunomics/processing/TCGA/processed_data/overall_read_depths_and_sample_information'),pattern="*table.txt",full.names = TRUE)){

  TABLE <-read.table(FILE,header=TRUE,sep='\t')
  names(TABLE)[names(TABLE)=='race'] <- 'race_self_reported'
  
  TABLE <- merge(TABLE,PCA_ANCESTRY, by='submitter_id')
  
  write.table(x=TABLE,file=gsub('.txt','_final.txt',FILE),sep='\t',row.names=FALSE,quote=FALSE)
}

# PART 2 - JOINT GENOTYPING ON BRCA TO DETERMINE MINOR ALLELE FREQUENCY, MINOR ALLELE CONCENTRATION IN GERMLINE AND SOMATIC TISSUE

## DOWNLOAD CAPTURE KITS AND CREATE BED OF COMMON CAPTURE REGION

### 1. Create bed file containing regions of interest

#### Option 1: Bed file contains all protein-coding regions

In [13]:
%%bash

#for i in {1..22}; do cat /research/bsi/projects/PI/tertiary/Asmann_Yan_wangy3/s212975.Wickland_Immunomics/gencode.v22.annotation.gtf | grep protein_coding | cut -f1,4,5 | grep -w chr${i} | sort | uniq | awk '{if($2!=$3) print}' > /research/bsi/projects/PI/tertiary/Asmann_Yan_wangy3/s212975.Wickland_Immunomics/processing/TCGA/beds/gencode.v22.annotation_chr${i}_protein-coding.bed;done;


#### Option 2: Bed file contains only the NimbleGen kit common capture regions
•  Bed file contains intersection of these 3 capture kits, lifted over to hg38 via UCSC tool:  
• https://sequencing.roche.com/en/support-resources/discontinued-products/seqcap-ez-exome-v3-kit.html  
•  https://web.archive.org/web/20161106062747/http://genome.wustl.edu/pub/custom_capture/hg18_nimblegen_exome_version_2/hg18_nimblegen_exome_version_2.bed  
•  https://web.archive.org/web/20141010064923/http://www.nimblegen.com/products/seqcap/ez/v2/index.html

In [3]:
%%bash

#sort by chrom, then start
for file in /research/bsi/projects/PI/tertiary/Asmann_Yan_wangy3/s212975.Wickland_Immunomics/processing/TCGA/processed_data/capture_kit/*hg38liftover.bed; do 

~/s212975.Wickland_Immunomics/tools/bedtools2/bin/bedtools sort -i ${file} > `dirname ${file}`/`basename ${file} .bed`_SORTED.bed;done;

In [9]:
%%bash

#run bedtools multiinter, keeping only those regions common to all three kits 
~/s212975.Wickland_Immunomics/tools/bedtools2/bin/bedtools multiinter -i /research/bsi/projects/PI/tertiary/Asmann_Yan_wangy3/s212975.Wickland_Immunomics/processing/TCGA/processed_data/capture_kit/SeqCap_EZ_Exome_v3_hg19_capture_targets_hg38liftover_SORTED.bed /research/bsi/projects/PI/tertiary/Asmann_Yan_wangy3/s212975.Wickland_Immunomics/processing/TCGA/processed_data/capture_kit/hg18_nimblegen_exome_version_2_hg38liftover_SORTED.bed  /research/bsi/projects/PI/tertiary/Asmann_Yan_wangy3/s212975.Wickland_Immunomics/processing/TCGA/processed_data/capture_kit/SeqCap_EZ_Exome_v2_hg38liftover_SORTED.bed | awk '$4 == 3 ' > /research/bsi/projects/PI/tertiary/Asmann_Yan_wangy3/s212975.Wickland_Immunomics/processing/TCGA/processed_data/capture_kit/three_capture_kits_intersection_hg38_liftover.bed



In [10]:
%%bash

#print capture size

cat /research/bsi/projects/PI/tertiary/Asmann_Yan_wangy3/s212975.Wickland_Immunomics/processing/TCGA/processed_data/capture_kit/SeqCap_EZ_Exome_v3_hg19_capture_targets_hg38liftover_SORTED.bed |  awk '{print $3-$2+1}' | awk '{sum += $1} END {print sum/1000000}'

cat /research/bsi/projects/PI/tertiary/Asmann_Yan_wangy3/s212975.Wickland_Immunomics/processing/TCGA/processed_data/capture_kit/hg18_nimblegen_exome_version_2_hg38liftover_SORTED.bed |  awk '{print $3-$2+1}' | awk '{sum += $1} END {print sum/1000000}'

cat /research/bsi/projects/PI/tertiary/Asmann_Yan_wangy3/s212975.Wickland_Immunomics/processing/TCGA/processed_data/capture_kit/SeqCap_EZ_Exome_v2_hg38liftover_SORTED.bed |  awk '{print $3-$2+1}' | awk '{sum += $1} END {print sum/1000000}'

cat /research/bsi/projects/PI/tertiary/Asmann_Yan_wangy3/s212975.Wickland_Immunomics/processing/TCGA/processed_data/capture_kit/three_capture_kits_intersection_hg38_liftover.bed |  awk '{print $3-$2+1}' | awk '{sum += $1} END {print sum/1000000}'

64.5468
35.9727
80.5869
31.5054


In [8]:
%%bash

#extract the common-capture positions from each chromosome
for i in {1..22}; do cat /research/bsi/projects/PI/tertiary/Asmann_Yan_wangy3/s212975.Wickland_Immunomics/processing/TCGA/processed_data/capture_kit/three_capture_kits_intersection_hg38_liftover.bed | grep -w chr${i} | sort | uniq | awk '{if($2!=$3) print}' > /research/bsi/projects/PI/tertiary/Asmann_Yan_wangy3/s212975.Wickland_Immunomics/processing/TCGA/beds/intersection_three_nimblegen_kits_hg38_DW_chr${i}.bed;done;

### CALCULATE READ DEPTH OF THESE BRCA SAMPLES WITHIN THIS CAPTURE REGION
- This will do both tumor and normal
- As before, in the overall depth analysis, including the sex chromosomes here

In [33]:
%%bash

#create directories for scripts and for outputs
if [ -d "/home/mayo/m187735/s212975.Wickland_Immunomics/processing/TCGA/bed_read_depths/BRCA_NEW" ]; then rm -r /home/mayo/m187735//s212975.Wickland_Immunomics/processing/TCGA/bed_read_depths/BRCA_NEW;
fi;

mkdir -p /home/mayo/m187735/s212975.Wickland_Immunomics/processing/TCGA/bed_read_depths/BRCA_NEW

if [ -d "/home/mayo/m187735/s212975.Wickland_Immunomics/commands_to_qsub/TCGA/bed_read_depths/BRCA_NEW" ];	then rm -r /home/mayo/m187735/s212975.Wickland_Immunomics/commands_to_qsub/TCGA/bed_read_depths/BRCA_NEW;
fi;

mkdir -p /home/mayo/m187735/s212975.Wickland_Immunomics/commands_to_qsub/TCGA/bed_read_depths/BRCA_NEW

#create scripts to run view (to count # of reads) and mpileup (to count # of bases, avg # of bases) in each bed entry region for each sample
grep mbleg  /research/bsi/projects/PI/tertiary/Asmann_Yan_wangy3/s212975.Wickland_Immunomics/processing/TCGA/processed_data/overall_read_depths_and_sample_information/BRCA_all_reads_exome_master_table_final.txt | cut -f6 | sort | uniq | while read bam; do OUTPUT_FILE='/home/mayo/m187735/s212975.Wickland_Immunomics/processing/TCGA/bed_read_depths/BRCA_NEW/'`basename ${bam} .bam`'_bed_read_depths.txt'; echo "rm "$OUTPUT_FILE"; cut -f1-3 /research/bsi/projects/PI/tertiary/Asmann_Yan_wangy3/s212975.Wickland_Immunomics/processing/TCGA/processed_data/capture_kit/three_capture_kits_intersection_hg38_liftover.bed |uniq | while read bed_entry; do REGION=\`echo \${bed_entry} | sed 's/ /:/' | sed 's/ /-/'\`; echo \${REGION} | tr \\\n \\\t>> "$OUTPUT_FILE";  /home/mayo/m187735/s212975.Wickland_Immunomics/tools/samtools-1.10/bin/samtools mpileup --incl-flags 2 --excl-flags 1024 --min-MQ 20 --min-BQ 0 --region \${REGION} "$bam" | awk '{sum += \$4} END {print sum \"|\" sum/NR}' | tr \\\n '|' >> "$OUTPUT_FILE"; /home/mayo/m187735/s212975.Wickland_Immunomics/tools/samtools-1.10/bin/samtools view -c -f 2 -F 1024 -q 20 "$bam" \${REGION} >> "$OUTPUT_FILE"; done" >> /home/mayo/m187735/s212975.Wickland_Immunomics/commands_to_qsub/TCGA/bed_read_depths/BRCA_NEW/SAMPLE_`basename $bam .bam`.qsub;done;


#submit qsubs
for file in /home/mayo/m187735/s212975.Wickland_Immunomics/commands_to_qsub/TCGA/bed_read_depths/BRCA_NEW/*qsub; do qsub -l h_vmem=2G -e /home/mayo/m187735/s212975.Wickland_Immunomics/commands_to_qsub/TCGA/bed_read_depths/BRCA_NEW/`basename ${file} .qsub`.err -o /home/mayo/m187735/s212975.Wickland_Immunomics/commands_to_qsub/TCGA/bed_read_depths/BRCA_NEW/`basename ${file} .qsub`.out -q 1-day,4-day,7-day,30-day -pe threaded 1 $file;done;

Your job 1142266 ("SAMPLE_0017ba4c33a07ba807b29140b0662cb1_gdc_realn.qsub") has been submitted
Your job 1142267 ("SAMPLE_003860a34c9b244a5d8435b220ca5673_gdc_realn.qsub") has been submitted
Your job 1142268 ("SAMPLE_0047eb6e338c99ed7e7e399875b83181_gdc_realn.qsub") has been submitted
Your job 1142269 ("SAMPLE_00925769611d88cb0379798246946580_gdc_realn.qsub") has been submitted
Your job 1142270 ("SAMPLE_00a0b7fe73741312c23f4f63317cc972_gdc_realn.qsub") has been submitted
Your job 1142271 ("SAMPLE_01218e1cd1b806450e6954adedb202b1_gdc_realn.qsub") has been submitted
Your job 1142272 ("SAMPLE_015537fb5f5fb8cdbefb1556ec7568e5_gdc_realn.qsub") has been submitted
Your job 1142273 ("SAMPLE_01ba6cd35f5da1da90e1c2d95eab0068_gdc_realn.qsub") has been submitted
Your job 1142274 ("SAMPLE_01c12736511241497f2e976c1764b1e4_gdc_realn.qsub") has been submitted
Your job 1142275 ("SAMPLE_01e5961cf18d86fb266ca104136b5361_gdc_realn.qsub") has been submitted
Your job 1142276 ("SAMPLE_0205cabb2bae0812e6966871

#### Make sure it's done

In [ ]:
%%bash

for file in /research/bsi/projects/PI/tertiary/Asmann_Yan_wangy3/s212975.Wickland_Immunomics/processing/TCGA/bed_read_depths/BRCA_NEW/*txt; do wc -l $file;done | cut -d ' ' -f1 | sort | uniq -c

##  Run mpileup on all positions within capture region
#### Note that coverage of 0 willl NOT be recorded

#### First load the master tables that are also loaded by the analysis_and_plots.R script

In [45]:
%%R

######################################
#MASTER TABLE
######################################

PLOT_BY_RACE_AND_SAMPLING_SITE <- function(MAPPED_READS_OR_ALL_READS,CANCER_TYPES,EXOME_OR_RNASEQ){
  
  if (CANCER_TYPES=='all'){
    CANCER_LIST <- c('BRCA','PRAD','LUAD','UCEC','KIRC','COAD','KIRP')
    
    #Create dataframe to hold master table for all cancers
    master_table_master <- data.frame()
    
    if(MAPPED_READS_OR_ALL_READS != 'mapped_BRCA_reads_within_nimblegen_capture'){
      
      #Load data
      for (CANCER_NAME in CANCER_LIST){
        
        #load overall depths data
        master_table <- read.table(paste0('/home/mayo/m187735/s212975.Wickland_Immunomics/processing/TCGA/processed_data/overall_read_depths_and_sample_information/',CANCER_NAME,'_',MAPPED_READS_OR_ALL_READS,'_',EXOME_OR_RNASEQ,'_master_table_final.txt'),header=TRUE,sep='\t')
        master_table_master <- rbind(master_table_master,master_table)
      }
    }
  }
  
  if (CANCER_TYPES != 'all'){
    CANCER_NAME <- CANCER_TYPES
    master_table_master <- read.table(paste0('/home/mayo/m187735/s212975.Wickland_Immunomics/processing/TCGA/processed_data/overall_read_depths_and_sample_information/',CANCER_NAME,'_',MAPPED_READS_OR_ALL_READS,'_',EXOME_OR_RNASEQ,'_master_table_final.txt'),header=TRUE,sep='\t')
    
    
  }
  
  
  #remove entries that have no exome data; some in dataframes have RNA-seq but no exome
  master_table_master <- subset(master_table_master, (bam_full_path != 'NA'))
  
  #set order for race 
  master_table_master <- subset(master_table_master, (race_PCA == 'White' | race_PCA == 'Black'))
  master_table_master$race_PCA <- factor(master_table_master$race_PCA, levels = c("White","Black"),ordered=TRUE)
  
  #rename tissue type and set order
  master_table_master$tissue.definition <- sub("Blood_Normal","Patient Germline",master_table_master$tissue.definition)
  master_table_master$tissue.definition <- sub("Primary_Tumor","Primary Tumor",master_table_master$tissue.definition)
  master_table_master$tissue.definition <- factor(master_table_master$tissue.definition, levels=c('Primary Tumor','Patient Germline'))
  master_table_master$Cancer <- factor(master_table_master$Cancer,levels=c('BRCA','PRAD','LUAD','UCEC','KIRC','COAD','KIRP'),ordered=TRUE)
  
  #if any capture kit listed multiple times for a patient, combine them into single row, so that no patient is counted more than once for same tissue type
  CAPTURE_KIT_FIXED <- master_table_master %>% group_by(submitter_id_long,tissue.definition) %>% dplyr::summarise(CAPTURE_KIT_FIXED = toString(CAPTURE_KIT))
  CAPTURE_KIT_FIXED$CAPTURE_KIT_FIXED <- gsub(", ","|",CAPTURE_KIT_FIXED$CAPTURE_KIT_FIXED)
  
  master_table_master <- merge(master_table_master, CAPTURE_KIT_FIXED, by=c('submitter_id_long','tissue.definition'))
  master_table_master <- master_table_master[(names(master_table_master)!='CAPTURE_KIT')]
  
  master_table_master <- unique(master_table_master)
  
  #simplify capture kit
    master_table_master$CAPTURE_KIT_simp <-  ifelse(grepl('hg18',master_table_master$CAPTURE_KIT_FIXED),'NimbleGen hg18 Exome v2',ifelse(grepl('v3.0',master_table_master$CAPTURE_KIT_FIXED),'NimbleGen SeqCap EZ Exome v3',ifelse(grepl('v2.0',master_table_master$CAPTURE_KIT_FIXED),'NimbleGen SeqCap EZ Exome v2',ifelse(grepl('V2.0',master_table_master$CAPTURE_KIT_FIXED),'NimbleGen SeqCap EZ Exome v2',ifelse(grepl('Custom',master_table_master$CAPTURE_KIT_FIXED),'Custom V2 Exome Bait',ifelse(grepl('Gapfiller',master_table_master$CAPTURE_KIT_FIXED),'Gapfiller_7m',ifelse(grepl('Rome',master_table_master$CAPTURE_KIT_FIXED),'Roche SeqCap EZ HGSC VCRome',ifelse(grepl('SureSelect',master_table_master$CAPTURE_KIT_FIXED),'Agilent SureSelect Human All Exon v2','Not reported')))))))) 
  
  
  #clean up capture kit and order according to date
  master_table_master$CAPTURE_KIT_simp <- gsub("SureSelect Human All Exon 38 Mb v2","Agilent SureSelect Human All Exon v2",master_table_master$CAPTURE_KIT_simp)
  
  
  #reorder capture kit simp
  master_table_master$CAPTURE_KIT_simp <- factor(master_table_master$CAPTURE_KIT_simp, levels=c('Agilent SureSelect Human All Exon v2','Custom V2 Exome Bait','NimbleGen hg18 Exome v2','NimbleGen SeqCap EZ Exome v2','NimbleGen SeqCap EZ Exome v3','Roche SeqCap EZ HGSC VCRome','Gapfiller_7m','Not reported'),ordered=TRUE)
  
  return(master_table_master)
}


ALL_READS_FOR_PLOTTING <- unique(PLOT_BY_RACE_AND_SAMPLING_SITE('all_reads','all','exome'))
MAPPED_READS_FOR_PLOTTING <- unique(PLOT_BY_RACE_AND_SAMPLING_SITE('mapped_reads','all','exome'))
UNMAPPED_READS_FOR_PLOTTING <- unique(PLOT_BY_RACE_AND_SAMPLING_SITE('unmapped_reads','all','exome'))

SeqCapV2_BRCA_READS_FOR_PLOTTING <- unique(PLOT_BY_RACE_AND_SAMPLING_SITE('mapped_reads_within_nimblegen_SeqCap_EZ_Exome_v2_hg38liftover_SORTED','BRCA','exome'))
SeqCapV3_BRCA_READS_FOR_PLOTTING <- unique(PLOT_BY_RACE_AND_SAMPLING_SITE('mapped_reads_within_nimblegen_SeqCap_EZ_Exome_v3_hg19_capture_targets_hg38liftover_SORTED','BRCA','exome'))
hg18_BRCA_READS_FOR_PLOTTING <- unique(PLOT_BY_RACE_AND_SAMPLING_SITE('mapped_reads_within_nimblegen_hg18_nimblegen_exome_version_2_hg38liftover_SORTED','BRCA','exome'))




##################################
#BRCA CAPTURE KITS -- KEEP ONLY THE SAMPLES CAPTURED BY THEIR RESPECTIVE KITS
##################################

#if sample called by two kits, just consider as having been captured by smallest
table(SeqCapV2_BRCA_READS_FOR_PLOTTING$CAPTURE_KIT_simp)
SeqCapV2_BRCA_READS_FOR_PLOTTING <- subset(SeqCapV2_BRCA_READS_FOR_PLOTTING, CAPTURE_KIT_simp == 'NimbleGen SeqCap EZ Exome v2')
SeqCapV3_BRCA_READS_FOR_PLOTTING <- subset(SeqCapV3_BRCA_READS_FOR_PLOTTING, CAPTURE_KIT_simp == 'NimbleGen SeqCap EZ Exome v3')
hg18_BRCA_READS_FOR_PLOTTING <- subset(hg18_BRCA_READS_FOR_PLOTTING, CAPTURE_KIT_simp == 'NimbleGen hg18 Exome v2')

BRCA_three_nimblegen_kits_overall_reads <- rbind(hg18_BRCA_READS_FOR_PLOTTING, SeqCapV2_BRCA_READS_FOR_PLOTTING,SeqCapV3_BRCA_READS_FOR_PLOTTING)



`summarise()` has grouped output by 'submitter_id_long'. You can override using the `.groups` argument.
`summarise()` has grouped output by 'submitter_id_long'. You can override using the `.groups` argument.
`summarise()` has grouped output by 'submitter_id_long'. You can override using the `.groups` argument.
`summarise()` has grouped output by 'submitter_id_long'. You can override using the `.groups` argument.
`summarise()` has grouped output by 'submitter_id_long'. You can override using the `.groups` argument.
`summarise()` has grouped output by 'submitter_id_long'. You can override using the `.groups` argument.


#### Then run mpileup

In [48]:
%%R

CAPTURE_FILE <- '/research/bsi/projects/PI/tertiary/Asmann_Yan_wangy3/s212975.Wickland_Immunomics/processing/TCGA/processed_data/capture_kit/three_capture_kits_intersection_hg38_liftover.bed'

OUTPUT_NAME <- 'All_genes'


for (row in 1:nrow(BRCA_three_nimblegen_kits_overall_reads)){
  BAM_PATH <- as.character(BRCA_three_nimblegen_kits_overall_reads[row,6])
    SAMPLE_ID <- as.character(BRCA_three_nimblegen_kits_overall_reads[row,3])
    RACE <- as.character(BRCA_three_nimblegen_kits_overall_reads[row,17])
    TISSUE <- as.character(BRCA_three_nimblegen_kits_overall_reads[row,2])
    CAPKIT <- as.character(BRCA_three_nimblegen_kits_overall_reads[row,19])
  
  system(paste0('if [ ! -d /research/bsi/projects/PI/tertiary/Asmann_Yan_wangy3/s212975.Wickland_Immunomics/processing/TCGA/processed_data/DP_per_position_from_bams/BRCA_NEW/',gsub(' ','_',TISSUE),'/',OUTPUT_NAME, ' ]; then mkdir -p /research/bsi/projects/PI/tertiary/Asmann_Yan_wangy3/s212975.Wickland_Immunomics/processing/TCGA/processed_data/DP_per_position_from_bams/BRCA_NEW/',gsub(' ','_',TISSUE),'/',OUTPUT_NAME,'; fi'))

 # system(paste0('echo "/research/bsi/projects/PI/tertiary/Asmann_Yan_wangy3/s212975.Wickland_Immunomics/tools/samtools-1.10/bin/samtools mpileup ',BAM_PATH,' --incl-flags 2 --excl-flags 1024 --min-MQ 20 --min-BQ 0 --positions ',CAPTURE_FILE ,' | cut -f1,2,4 > ',paste0('/research/bsi/projects/PI/tertiary/Asmann_Yan_wangy3/s212975.Wickland_Immunomics/processing/TCGA/processed_data/DP_per_position_from_bams/BRCA/',gsub(' ','_',TISSUE),'/',OUTPUT_NAME,'/',SAMPLE_ID,'_',RACE,'_',gsub(' ','_',TISSUE),'.txt" | qsub -l h_vmem=2G -e ',paste0("/research/bsi/projects/PI/tertiary/Asmann_Yan_wangy3/s212975.Wickland_Immunomics/processing/TCGA/processed_data/DP_per_position_from_bams/BRCA/",gsub(" ","_",TISSUE),"/",OUTPUT_NAME,'/',SAMPLE_ID,"_",RACE,"_",gsub(" ","_",TISSUE),".err "),paste0("-o /research/bsi/projects/PI/tertiary/Asmann_Yan_wangy3/s212975.Wickland_Immunomics/processing/TCGA/processed_data/DP_per_position_from_bams/BRCA/",gsub(" ","_",TISSUE),"/",OUTPUT_NAME,'/',SAMPLE_ID,"_",RACE,"_",gsub(" ","_",TISSUE)),'.out -q 1-day,4-day,7-day,30-day -pe threaded 1 -N ',SAMPLE_ID)))
  
  OUTPUT_FILE=paste0('/research/bsi/projects/PI/tertiary/Asmann_Yan_wangy3/s212975.Wickland_Immunomics/processing/TCGA/processed_data/DP_per_position_from_bams/BRCA_NEW/',gsub(' ','_',TISSUE),'/',OUTPUT_NAME,'/',SAMPLE_ID,'_',RACE,'_',gsub(' ','_',TISSUE),'_',gsub(' ','_',CAPKIT),'.txt')

  system(paste0('echo "/research/bsi/projects/PI/tertiary/Asmann_Yan_wangy3/s212975.Wickland_Immunomics/tools/samtools-1.10/bin/samtools mpileup ',BAM_PATH,' --incl-flags 2 --excl-flags 1024 --min-MQ 20 --min-BQ 0 --positions ',CAPTURE_FILE ,' | cut -f1,2,4 > ',OUTPUT_FILE,' " | qsub -l h_vmem=2G -e /research/bsi/projects/PI/tertiary/Asmann_Yan_wangy3/s212975.Wickland_Immunomics/processing/TCGA/processed_data/DP_per_position_from_bams/BRCA_NEW/',gsub(" ","_",TISSUE),'/',OUTPUT_NAME,'/',SAMPLE_ID,'_',RACE,'_',gsub(' ','_',TISSUE),'.err -o /research/bsi/projects/PI/tertiary/Asmann_Yan_wangy3/s212975.Wickland_Immunomics/processing/TCGA/processed_data/DP_per_position_from_bams/BRCA_NEW/',gsub(' ','_',TISSUE),'/',OUTPUT_NAME,'/',SAMPLE_ID,'_',RACE,'_',gsub(' ','_',TISSUE),'.out -q 1-day,4-day,7-day,30-day -pe threaded 1 -N ',SAMPLE_ID))
  
}
#note that for v3 kit, for some reason file suffix not added





#### Generate summary file of results from above -- depth within NimbleGen capture region (wait...why do I need this, again, if I'm using the more raw measurement in Fig 2, of just dividing overall by the capture size?)

In [49]:
%%bash

for TISSUE in Primary_Tumor Patient_Germline; do 

    RESULTS_DIREC=/research/bsi/projects/PI/tertiary/Asmann_Yan_wangy3/s212975.Wickland_Immunomics/processing/TCGA/processed_data/DP_per_position_from_bams/BRCA_NEW/${TISSUE}/All_genes/

    for RESULTS_FILE in ${RESULTS_DIREC}/TCGA*txt;
        do echo "echo -e 'SAMPLE\tRACE\tTISSUE\tCAPTURE_KIT\tPOSITIONS_WITH_COVERAGE_1_TO_10\tPOSITIONS_WITH_COVERAGE_11_TO_39\tPOSITIONS_WITH_COVERAGE_40_PLUS' > ${RESULTS_DIREC}/`basename ${RESULTS_FILE} .txt`.summary;

        echo `basename ${RESULTS_FILE}` | cut -d '_' -f1 | tr '\n' '\t' >> ${RESULTS_DIREC}/`basename ${RESULTS_FILE} .txt`.summary;
        echo `basename ${RESULTS_FILE}` | cut -d '_' -f2 | tr '\n' '\t' >> ${RESULTS_DIREC}/`basename ${RESULTS_FILE} .txt`.summary;
        echo `basename ${RESULTS_FILE}` | cut -d '_' -f3,4 | sed 's/.txt//g' | tr '\n' '\t' >> ${RESULTS_DIREC}/`basename ${RESULTS_FILE} .txt`.summary;
        echo `basename ${RESULTS_FILE}` | cut -d '_' -f5- | sed 's/.txt//g' | tr '\n' '\t' >> ${RESULTS_DIREC}/`basename ${RESULTS_FILE} .txt`.summary;
        cat ${RESULTS_FILE} | awk  '\$3 >=1 && \$3 <= 10'  | wc -l | tr '\n' '\t' >> ${RESULTS_DIREC}/`basename ${RESULTS_FILE} .txt`.summary;
        cat ${RESULTS_FILE} | awk  '\$3 >=11 && \$3 <= 39'  | wc -l | tr '\n' '\t' >> ${RESULTS_DIREC}/`basename ${RESULTS_FILE} .txt`.summary;
        cat ${RESULTS_FILE} | awk  '\$3 >=40'  | wc -l | tr -d '\t' >> ${RESULTS_DIREC}/`basename ${RESULTS_FILE} .txt`.summary" | qsub -l h_vmem=500M -N `basename ${RESULTS_FILE} .txt` -q 1-day,4-day,7-day,30-day -pe threaded 1  ;done;done;
		

Your job 1170158 ("TCGA-3C-AAAU_White_Primary_Tumor_NimbleGen_SeqCap_EZ_Exome_v3") has been submitted
Your job 1170159 ("TCGA-3C-AALI_Black_Primary_Tumor_NimbleGen_SeqCap_EZ_Exome_v3") has been submitted
Your job 1170160 ("TCGA-3C-AALJ_Black_Primary_Tumor_NimbleGen_SeqCap_EZ_Exome_v3") has been submitted
Your job 1170161 ("TCGA-3C-AALK_Black_Primary_Tumor_NimbleGen_SeqCap_EZ_Exome_v3") has been submitted
Your job 1170162 ("TCGA-4H-AAAK_White_Primary_Tumor_NimbleGen_SeqCap_EZ_Exome_v3") has been submitted
Your job 1170163 ("TCGA-5L-AAT0_White_Primary_Tumor_NimbleGen_SeqCap_EZ_Exome_v3") has been submitted
Your job 1170164 ("TCGA-5L-AAT1_White_Primary_Tumor_NimbleGen_SeqCap_EZ_Exome_v3") has been submitted
Your job 1170165 ("TCGA-5T-A9QA_Black_Primary_Tumor_NimbleGen_SeqCap_EZ_Exome_v3") has been submitted
Your job 1170166 ("TCGA-A1-A0SD_White_Primary_Tumor_NimbleGen_hg18_Exome_v2") has been submitted
Your job 1170167 ("TCGA-A1-A0SE_White_Primary_Tumor_NimbleGen_hg18_Exome_v2") has been 

#### Combine summary files (can probably delete this -- seems not to have been used)

In [55]:

%%R

OUTPUT_NAME <- 'All_genes'

ALL_SAMPLES <- data.frame()

for (TISSUE in c('Primary_Tumor','Patient_Germline')){
    for (FILE in list.files(paste0('/research/bsi/projects/PI/tertiary/Asmann_Yan_wangy3/s212975.Wickland_Immunomics/processing/TCGA/processed_data/DP_per_position_from_bams/BRCA_NEW/',TISSUE,'/',OUTPUT_NAME),pattern="*summary*",full.names = TRUE)){
      SAMPLE <- read.table(FILE,header=TRUE,sep='\t')
      ALL_SAMPLES <- rbind(ALL_SAMPLES,SAMPLE)

    }
}

#get total capture region size
common_capture_region <- read.table('/research/bsi/projects/PI/tertiary/Asmann_Yan_wangy3/s212975.Wickland_Immunomics/processing/TCGA/processed_data/capture_kit/three_capture_kits_intersection_hg38_liftover.bed',header=FALSE,sep='\t')

common_capture_region_size=sum(common_capture_region[,3]-common_capture_region[,2]+1)

ALL_SAMPLES$TOTAL_POSITIONS_IN_CAPTURE_REGION <- common_capture_region_size

ALL_SAMPLES$POSITIONS_WITH_NO_COVERAGE <- ALL_SAMPLES$TOTAL_POSITIONS_IN_CAPTURE_REGION-(ALL_SAMPLES$POSITIONS_WITH_COVERAGE_1_TO_10+ALL_SAMPLES$POSITIONS_WITH_COVERAGE_11_TO_39+ALL_SAMPLES$POSITIONS_WITH_COVERAGE_40_PLUS)

ALL_SAMPLES_TUMOR <- subset(ALL_SAMPLES,TISSUE=='Primary_Tumor') 
ALL_SAMPLES_GERMLINE <- subset(ALL_SAMPLES,TISSUE=='Paient_Germline')

write.table(ALL_SAMPLES_TUMOR,'/research/bsi/projects/PI/tertiary/Asmann_Yan_wangy3/s212975.Wickland_Immunomics/processing/TCGA/processed_data/DP_per_position_from_bams/BRCA_NEW/Primary_Tumor/All_genes/SUMMARY_DEPTHS_ALL_SAMPLES_IN_CAPTURE_REGION.txt',sep='\t',quote=FALSE,row.names=FALSE)
write.table(ALL_SAMPLES_GERMLINE,'/research/bsi/projects/PI/tertiary/Asmann_Yan_wangy3/s212975.Wickland_Immunomics/processing/TCGA/processed_data/DP_per_position_from_bams/BRCA_NEW/Patient_Germline/All_genes/SUMMARY_DEPTHS_ALL_SAMPLES_IN_CAPTURE_REGION.txt',sep='\t',quote=FALSE,row.names=FALSE)
    

### 2. Run HaplotypeCaller on TCGA bams

In [7]:
%%bash

for i in {1..22}; do 

    #type of cancer (e.g. LUAD, ${CANCER_NAME}, PRAD, COAD)
    TISSUE=Blood_Normal
    CANCER_NAME=BRCA
    BED_FILE=/research/bsi/projects/PI/tertiary/Asmann_Yan_wangy3/s212975.Wickland_Immunomics/processing/TCGA/beds/intersection_three_nimblegen_kits_hg38_DW_chr${i}.bed
    PROJECT=nimblegen_capture_NOV19_chr${i}

    #create directories for scripts and for outputs
    mkdir -p /home/mayo/m187735/s212975.Wickland_Immunomics/processing/TCGA/bams_lists_hg38/${PROJECT}
    mkdir -p /home/mayo/m187735/s212975.Wickland_Immunomics/processing/TCGA/processed_data/HaplotypeCaller_on_TCGA_bams/${PROJECT}/${CANCER_NAME}/${TISSUE}/White
    mkdir -p /home/mayo/m187735/s212975.Wickland_Immunomics/processing/TCGA/processed_data/HaplotypeCaller_on_TCGA_bams/${PROJECT}/${CANCER_NAME}/${TISSUE}/Black

    if [ -d "/home/mayo/m187735/s212975.Wickland_Immunomics/commands_to_qsub/TCGA/HaplotypeCaller_on_TCGA_bams/${PROJECT}/${CANCER_NAME}/${TISSUE}/" ];
    then rm -r /home/mayo/m187735/s212975.Wickland_Immunomics/commands_to_qsub/TCGA/HaplotypeCaller_on_TCGA_bams/${PROJECT}/${CANCER_NAME}/${TISSUE}/;
    fi;

    mkdir -p /home/mayo/m187735/s212975.Wickland_Immunomics/commands_to_qsub/TCGA/HaplotypeCaller_on_TCGA_bams/${PROJECT}/${CANCER_NAME}/${TISSUE}/


    #list all hg38 bams on mforge for cancer type
    echo " " > /home/mayo/m187735/s212975.Wickland_Immunomics/processing/TCGA/bams_lists_hg38/${PROJECT}/${CANCER_NAME}_bam_list_${TISSUE}.txt #first create file with blank line at top, otherwise skips when generating commands for qsub files...

    #only want samples captured by a Nimblegen kit
    if [ "$TISSUE" = "Primary_Tumor" ]; 
    then cat /research/bsi/projects/PI/tertiary/Asmann_Yan_wangy3/s212975.Wickland_Immunomics/processing/TCGA/processed_data/overall_read_depths_and_sample_information/${CANCER_NAME}_all_reads_exome_master_table_final.txt | awk '{print $1,$18,$6,$4,$16}' FS='\t' OFS='\t' | awk '$3 != "NA" && $4 == "Primary_Tumor" && $5 ~ "imble"' FS='\t' >> /home/mayo/m187735/s212975.Wickland_Immunomics/processing/TCGA/bams_lists_hg38/${PROJECT}/${CANCER_NAME}_bam_list_${TISSUE}.txt;

    elif [ "$TISSUE" = "Blood_Normal" ]; 
    then cat /research/bsi/projects/PI/tertiary/Asmann_Yan_wangy3/s212975.Wickland_Immunomics/processing/TCGA/processed_data/overall_read_depths_and_sample_information/${CANCER_NAME}_all_reads_exome_master_table_final.txt | awk '{print $1,$18,$6,$4,$16}' FS='\t' OFS='\t' | awk '$3 != "NA" && $4 == "Blood_Normal" && $5 ~ "imble"' FS='\t' >> /home/mayo/m187735/s212975.Wickland_Immunomics/processing/TCGA/bams_lists_hg38/${PROJECT}/${CANCER_NAME}_bam_list_${TISSUE}.txt;
    fi


    #create scripts for sets of 10 samples
    TOTAL_NUMBER_OF_BAMS=`wc -l /home/mayo/m187735/s212975.Wickland_Immunomics/processing/TCGA/bams_lists_hg38/${PROJECT}/${CANCER_NAME}_bam_list_${TISSUE}.txt | cut -d ' ' -f1`
    NUMBER_OF_SPLITS=`echo $((TOTAL_NUMBER_OF_BAMS/10))`
    BAM_LOOP_END=`echo $((NUMBER_OF_SPLITS*10))`


    COUNTER=1
    COUNTER2=11
    #the positional variables in awk correspond to the order of cols in the bam_list.txt generated above
    until [ ${COUNTER} -eq $(($BAM_LOOP_END+11)) ];  do awk -v a=${COUNTER} -v b=${COUNTER2} "NR==a,NR==b" /home/mayo/m187735/s212975.Wickland_Immunomics/processing/TCGA/bams_lists_hg38/${PROJECT}/${CANCER_NAME}_bam_list_${TISSUE}.txt | while read line; do awk -v PROJECT=${PROJECT}  -v BED_FILE=${BED_FILE} '{print "java -jar /research/bsi/projects/PI/tertiary/Asmann_Yan_wangy3/s212975.Wickland_Immunomics/tools/GenomeAnalysisTK.jar -T HaplotypeCaller -R /research/bsi/projects/PI/tertiary/Asmann_Yan_wangy3/s212975.Wickland_Immunomics/GRCh38.d1.vd1.fa -I "$3" -ERC GVCF --intervals "BED_FILE" -o  /home/mayo/m187735/s212975.Wickland_Immunomics/processing/TCGA/processed_data/HaplotypeCaller_on_TCGA_bams/"PROJECT"/${CANCER_NAME}/"$4"/"$2"/"$1".g.vcf"}' OFS='\t' > /home/mayo/m187735/s212975.Wickland_Immunomics/commands_to_qsub/TCGA/HaplotypeCaller_on_TCGA_bams/${PROJECT}/${CANCER_NAME}/${TISSUE}/samples_${COUNTER}-${COUNTER2}.qsub;done;COUNTER=`echo $((${COUNTER}+10))`;COUNTER2=`echo $((${COUNTER}+10))`;done;

    #submit qsubs
    for file in /research/bsi/projects/PI/tertiary/Asmann_Yan_wangy3/s212975.Wickland_Immunomics/commands_to_qsub/TCGA/HaplotypeCaller_on_TCGA_bams/${PROJECT}/${CANCER_NAME}/${TISSUE}/*qsub; do qsub -l h_vmem=10G -e /home/mayo/m187735/s212975.Wickland_Immunomics/commands_to_qsub/TCGA/HaplotypeCaller_on_TCGA_bams/${PROJECT}/${CANCER_NAME}/${TISSUE}/`basename ${file} .qsub`.err -o /home/mayo/m187735/s212975.Wickland_Immunomics/commands_to_qsub/TCGA/HaplotypeCaller_on_TCGA_bams/${PROJECT}/${CANCER_NAME}/${TISSUE}/`basename ${file} .qsub`.out -pe threaded 4 -q 1-day,4-day,7-day,30-day $file;done;
done;




Your job 1149367 ("samples_1-11.qsub") has been submitted
Your job 1149368 ("samples_101-111.qsub") has been submitted
Your job 1149369 ("samples_11-21.qsub") has been submitted
Your job 1149370 ("samples_111-121.qsub") has been submitted
Your job 1149371 ("samples_121-131.qsub") has been submitted
Your job 1149372 ("samples_131-141.qsub") has been submitted
Your job 1149373 ("samples_141-151.qsub") has been submitted
Your job 1149374 ("samples_151-161.qsub") has been submitted
Your job 1149375 ("samples_161-171.qsub") has been submitted
Your job 1149376 ("samples_171-181.qsub") has been submitted
Your job 1149377 ("samples_181-191.qsub") has been submitted
Your job 1149378 ("samples_191-201.qsub") has been submitted
Your job 1149379 ("samples_201-211.qsub") has been submitted
Your job 1149380 ("samples_21-31.qsub") has been submitted
Your job 1149381 ("samples_211-221.qsub") has been submitted
Your job 1149382 ("samples_221-231.qsub") has been submitted
Your job 1149383 ("samples_231-

#### Check number of files output

In [8]:
%%bash

PROJECT=nimblegen_capture_NOV19
CANCER_NAME=BRCA

for TISSUE in  Blood_Normal
do
    for i in {1..22}; do echo "chr"${i} ${TISSUE}; ls /research/bsi/projects/PI/tertiary/Asmann_Yan_wangy3/s212975.Wickland_Immunomics/processing/TCGA/processed_data/HaplotypeCaller_on_TCGA_bams/${PROJECT}_chr${i}/${CANCER_NAME}/${TISSUE}/White/*idx | wc -l;ls /research/bsi/projects/PI/tertiary/Asmann_Yan_wangy3/s212975.Wickland_Immunomics/processing/TCGA/processed_data/HaplotypeCaller_on_TCGA_bams/${PROJECT}_chr${i}/${CANCER_NAME}/${TISSUE}/Black/*idx | wc -l; done;
done

chr1 Blood_Normal
581
161
chr2 Blood_Normal
581
161
chr3 Blood_Normal
581
161
chr4 Blood_Normal
581
161
chr5 Blood_Normal
581
161
chr6 Blood_Normal
581
161
chr7 Blood_Normal
581
161
chr8 Blood_Normal
581
161
chr9 Blood_Normal
581
161
chr10 Blood_Normal
581
161
chr11 Blood_Normal
581
161
chr12 Blood_Normal
581
161
chr13 Blood_Normal
581
161
chr14 Blood_Normal
581
161
chr15 Blood_Normal
581
161
chr16 Blood_Normal
581
161
chr17 Blood_Normal
581
161
chr18 Blood_Normal
581
161
chr19 Blood_Normal
581
161
chr20 Blood_Normal
581
161
chr21 Blood_Normal
581
161
chr22 Blood_Normal
581
161


### 3. Run CombineGVCFs on GVCFs

In [11]:
%%bash


for i in {1..22}; do 
    
    PROJECT=nimblegen_capture_NOV19_chr${i}
    CANCER_NAME=BRCA
    TISSUE=Blood_Normal


    #create directory for output
    mkdir /research/bsi/projects/PI/tertiary/Asmann_Yan_wangy3/s212975.Wickland_Immunomics/processing/TCGA/processed_data/HaplotypeCaller_on_TCGA_bams/${PROJECT}/${CANCER_NAME}/${TISSUE}/combined

    #list all8 GVCFs
    ls /home/mayo/m187735/s212975.Wickland_Immunomics/processing/TCGA/processed_data/HaplotypeCaller_on_TCGA_bams/${PROJECT}/${CANCER_NAME}/${TISSUE}/*/*vcf > /home/mayo/m187735/s212975.Wickland_Immunomics/processing/TCGA/bams_lists_hg38/${PROJECT}/${CANCER_NAME}_${TISSUE}_gvcf_list.txt

    grep ${TISSUE} /home/mayo/m187735/s212975.Wickland_Immunomics/processing/TCGA/bams_lists_hg38/${PROJECT}/${CANCER_NAME}_${TISSUE}_gvcf_list.txt | grep "White" > /home/mayo/m187735/s212975.Wickland_Immunomics/processing/TCGA/bams_lists_hg38/${PROJECT}/${CANCER_NAME}_gvcf_list_${TISSUE}_White.txt

    grep ${TISSUE} /home/mayo/m187735/s212975.Wickland_Immunomics/processing/TCGA/bams_lists_hg38/${PROJECT}/${CANCER_NAME}_${TISSUE}_gvcf_list.txt | grep "Black" > /home/mayo/m187735/s212975.Wickland_Immunomics/processing/TCGA/bams_lists_hg38/${PROJECT}/${CANCER_NAME}_gvcf_list_${TISSUE}_Black.txt



    for GROUP in {${TISSUE}_White,${TISSUE}_Black};
     do

    #create scripts for sets of 50 samples
    TOTAL_NUMBER_OF_GVCFS=`wc -l /home/mayo/m187735/s212975.Wickland_Immunomics/processing/TCGA/bams_lists_hg38/${PROJECT}/${CANCER_NAME}_gvcf_list_${GROUP}.txt | cut -d ' ' -f1`
    NUMBER_OF_SPLITS=`echo $((TOTAL_NUMBER_OF_GVCFS/50))`
    GVCF_LOOP_END=`echo $((NUMBER_OF_SPLITS*50))`


    COUNTER=1
    COUNTER2=50

     #the positional variables in awk correspond to the order of cols in the GVCF_list.txt generated above
     #group gvcfs into sets of 50, and use list of samples/paths in that group as parameter to CombineGVCFs
     #use tr to remove newline characters in the lists, replacing all except the last one with '--variant'

     until [ ${COUNTER} -eq $(($GVCF_LOOP_END+51)) ]; do echo -e "java -Xmx15g -jar /research/bsi/projects/PI/tertiary/Asmann_Yan_wangy3/s212975.Wickland_Immunomics/tools/GenomeAnalysisTK.jar -T CombineGVCFs -R /research/bsi/projects/PI/tertiary/Asmann_Yan_wangy3/s212975.Wickland_Immunomics/GRCh38.d1.vd1.fa --variant " | tr '\n' ' ' > /home/mayo/m187735/s212975.Wickland_Immunomics/commands_to_qsub/TCGA/HaplotypeCaller_on_TCGA_bams/${PROJECT}/${CANCER_NAME}/${TISSUE}/combined_${GROUP}_${COUNTER}-${COUNTER2}.qsub; awk -v a=${COUNTER} -v b=${COUNTER2} "NR==a,NR==b" /home/mayo/m187735/s212975.Wickland_Immunomics/processing/TCGA/bams_lists_hg38/${PROJECT}/${CANCER_NAME}_gvcf_list_${GROUP}.txt | tr '\n' '\t' | sed '$s/\t$/ /' | sed 's/\t/ --variant /g' >> /home/mayo/m187735/s212975.Wickland_Immunomics/commands_to_qsub/TCGA/HaplotypeCaller_on_TCGA_bams/${PROJECT}/${CANCER_NAME}/${TISSUE}/combined_${GROUP}_${COUNTER}-${COUNTER2}.qsub; echo "-o /home/mayo/m187735/s212975.Wickland_Immunomics/processing/TCGA/processed_data/HaplotypeCaller_on_TCGA_bams/${PROJECT}/${CANCER_NAME}/${TISSUE}/combined/${GROUP}_samples_${COUNTER}-${COUNTER2}.g.vcf" >> /home/mayo/m187735/s212975.Wickland_Immunomics/commands_to_qsub/TCGA/HaplotypeCaller_on_TCGA_bams/${PROJECT}/${CANCER_NAME}/${TISSUE}/combined_${GROUP}_${COUNTER}-${COUNTER2}.qsub;COUNTER=`echo $((${COUNTER}+50))`;COUNTER2=`echo $((${COUNTER}+49))`;done;
    done


    rm /research/bsi/projects/PI/tertiary/Asmann_Yan_wangy3/s212975.Wickland_Immunomics/commands_to_qsub/TCGA/HaplotypeCaller_on_TCGA_bams/${PROJECT}/${CANCER_NAME}/${TISSUE}/*combined*err 2> /dev/null
    rm /research/bsi/projects/PI/tertiary/Asmann_Yan_wangy3/s212975.Wickland_Immunomics/commands_to_qsub/TCGA/HaplotypeCaller_on_TCGA_bams/${PROJECT}/${CANCER_NAME}/${TISSUE}/*combined*out 2> /dev/null

    for file in /research/bsi/projects/PI/tertiary/Asmann_Yan_wangy3/s212975.Wickland_Immunomics/commands_to_qsub/TCGA/HaplotypeCaller_on_TCGA_bams/${PROJECT}/${CANCER_NAME}/${TISSUE}/combined*qsub; do qsub -l h_vmem=25G -e /home/mayo/m187735/s212975.Wickland_Immunomics/commands_to_qsub/TCGA/HaplotypeCaller_on_TCGA_bams/${PROJECT}/${CANCER_NAME}/${TISSUE}/`basename ${file} .qsub`.err -o /home/mayo/m187735/s212975.Wickland_Immunomics/commands_to_qsub/TCGA/HaplotypeCaller_on_TCGA_bams/${PROJECT}/${CANCER_NAME}/${TISSUE}/`basename ${file} .qsub`.out -q 1-day,4-day $file;done;
done





Your job 1161193 ("combined_Blood_Normal_Black_1-50.qsub") has been submitted
Your job 1161194 ("combined_Blood_Normal_Black_101-150.qsub") has been submitted
Your job 1161195 ("combined_Blood_Normal_Black_151-200.qsub") has been submitted
Your job 1161196 ("combined_Blood_Normal_Black_51-100.qsub") has been submitted
Your job 1161197 ("combined_Blood_Normal_White_1-50.qsub") has been submitted
Your job 1161198 ("combined_Blood_Normal_White_101-150.qsub") has been submitted
Your job 1161199 ("combined_Blood_Normal_White_151-200.qsub") has been submitted
Your job 1161200 ("combined_Blood_Normal_White_201-250.qsub") has been submitted
Your job 1161201 ("combined_Blood_Normal_White_251-300.qsub") has been submitted
Your job 1161202 ("combined_Blood_Normal_White_301-350.qsub") has been submitted
Your job 1161203 ("combined_Blood_Normal_White_351-400.qsub") has been submitted
Your job 1161204 ("combined_Blood_Normal_White_401-450.qsub") has been submitted
Your job 1161205 ("combined_Blood_N

#### Check number of files output

In [16]:
%%bash

PROJECT=nimblegen_capture_NOV19
CANCER_NAME=BRCA

for TISSUE in Blood_Normal
do
	for i in {1..22}; do echo "chr"${i} ${TISSUE}; ls /research/bsi/projects/PI/tertiary/Asmann_Yan_wangy3/s212975.Wickland_Immunomics/processing/TCGA/processed_data/HaplotypeCaller_on_TCGA_bams/${PROJECT}_chr${i}/${CANCER_NAME}/${TISSUE}/combined/*vcf |wc -l ;ls /research/bsi/projects/PI/tertiary/Asmann_Yan_wangy3/s212975.Wickland_Immunomics/processing/TCGA/processed_data/HaplotypeCaller_on_TCGA_bams/${PROJECT}_chr${i}/${CANCER_NAME}/${TISSUE}/combined/*idx | wc -l;done;
done

chr1 Blood_Normal
16
16
chr2 Blood_Normal
16
16
chr3 Blood_Normal
16
16
chr4 Blood_Normal
16
16
chr5 Blood_Normal
16
16
chr6 Blood_Normal
16
16
chr7 Blood_Normal
16
16
chr8 Blood_Normal
16
16
chr9 Blood_Normal
16
16
chr10 Blood_Normal
16
16
chr11 Blood_Normal
16
16
chr12 Blood_Normal
16
16
chr13 Blood_Normal
16
16
chr14 Blood_Normal
16
16
chr15 Blood_Normal
16
16
chr16 Blood_Normal
16
16
chr17 Blood_Normal
16
16
chr18 Blood_Normal
16
16
chr19 Blood_Normal
16
16
chr20 Blood_Normal
16
16
chr21 Blood_Normal
16
16
chr22 Blood_Normal
16
16


### 4. Run GenotypeGVCFs on combined GVCFs

In [19]:
%%bash


for i in {1..22}; do 

    CANCER_NAME=BRCA
    TISSUE=Blood_Normal
    PROJECT=nimblegen_capture_NOV19_chr${i}

    #create directory for output
    OUTPUT_DIR=/research/bsi/projects/PI/tertiary/Asmann_Yan_wangy3/s212975.Wickland_Immunomics/processing/TCGA/processed_data/GenotypeGVCFs_on_TCGA/${PROJECT}/${CANCER_NAME}/${TISSUE}

    if [ -d "/home/mayo/m187735/s212975.Wickland_Immunomics/commands_to_qsub/TCGA/HaplotypeCaller_on_TCGA_bams/${PROJECT}/${CANCER_NAME}/${TISSUE}/" ];
    then rm -r ${OUTPUT_DIR};
    fi;

    mkdir -p $OUTPUT_DIR
    mkdir -p /home/mayo/m187735/s212975.Wickland_Immunomics/commands_to_qsub/TCGA/GenotypeGVCFs_on_TCGA/${PROJECT}/${CANCER_NAME}/${TISSUE}

    INPUT_DIR=/research/bsi/projects/PI/tertiary/Asmann_Yan_wangy3/s212975.Wickland_Immunomics/processing/TCGA/processed_data/HaplotypeCaller_on_TCGA_bams/${PROJECT}/${CANCER_NAME}/${TISSUE}/combined/

    #for GROUP in {${TISSUE}_Black,${TISSUE}_White};
    #	do 
        #	for GVCF in ${INPUT_DIR}/${GROUP}_samples*g.vcf; do GVCFS_LIST="${GVCFS_LIST} --variant ${GVCF}"; done
            for GVCF in ${INPUT_DIR}/*samples*g.vcf; do GVCFS_LIST="${GVCFS_LIST} --variant ${GVCF}"; done

            #echo "java -Xmx20g -jar /research/bsi/projects/PI/tertiary/Asmann_Yan_wangy3/s212975.Wickland_Immunomics/tools/GenomeAnalysisTK.jar -T GenotypeGVCFs -R /research/bsi/projects/PI/tertiary/Asmann_Yan_wangy3/s212975.Wickland_Immunomics/GRCh38.d1.vd1.fa ${GVCFS_LIST} -o ${OUTPUT_DIR}/${GROUP}.vcf --disable_auto_index_creation_and_locking_when_reading_rods -nt 24" > /home/mayo/m187735/s212975.Wickland_Immunomics/commands_to_qsub/TCGA/GenotypeGVCFs_on_TCGA/${PROJECT}/${CANCER_NAME}/${TISSUE}/${GROUP}.qsub
            echo "java -Xmx20g -jar /research/bsi/projects/PI/tertiary/Asmann_Yan_wangy3/s212975.Wickland_Immunomics/tools/GenomeAnalysisTK.jar -T GenotypeGVCFs -R /research/bsi/projects/PI/tertiary/Asmann_Yan_wangy3/s212975.Wickland_Immunomics/GRCh38.d1.vd1.fa ${GVCFS_LIST} -o ${OUTPUT_DIR}/BlackWhite.vcf --disable_auto_index_creation_and_locking_when_reading_rods -nt 24" > /home/mayo/m187735/s212975.Wickland_Immunomics/commands_to_qsub/TCGA/GenotypeGVCFs_on_TCGA/${PROJECT}/${CANCER_NAME}/${TISSUE}/BlackWhite.qsub
            GVCFS_LIST=""
    #done


    rm /research/bsi/projects/PI/tertiary/Asmann_Yan_wangy3/s212975.Wickland_Immunomics/commands_to_qsub/TCGA/GenotypeGVCFs_on_TCGA/${PROJECT}/${CANCER_NAME}/${TISSUE}/*err 2> /dev/null 
    rm /research/bsi/projects/PI/tertiary/Asmann_Yan_wangy3/s212975.Wickland_Immunomics/commands_to_qsub/TCGA/GenotypeGVCFs_on_TCGA/${PROJECT}/${CANCER_NAME}/${TISSUE}/*out 2> /dev/null 

    for file in /research/bsi/projects/PI/tertiary/Asmann_Yan_wangy3/s212975.Wickland_Immunomics/commands_to_qsub/TCGA/GenotypeGVCFs_on_TCGA/${PROJECT}/${CANCER_NAME}/${TISSUE}/*qsub; do qsub -l h_vmem=20G -e /home/mayo/m187735/s212975.Wickland_Immunomics/commands_to_qsub/TCGA/GenotypeGVCFs_on_TCGA/${PROJECT}/${CANCER_NAME}/${TISSUE}/`basename ${file} .qsub`.err -o /home/mayo/m187735/s212975.Wickland_Immunomics/commands_to_qsub/TCGA/GenotypeGVCFs_on_TCGA/${PROJECT}/${CANCER_NAME}/${TISSUE}/`basename ${file} .qsub`.out -q 4-day -pe threaded 24 -N chr${i}_GenotypeGVCFs $file;done;
done

Your job 1161714 ("chr1_GenotypeGVCFs") has been submitted
Your job 1161715 ("chr2_GenotypeGVCFs") has been submitted
Your job 1161716 ("chr3_GenotypeGVCFs") has been submitted
Your job 1161717 ("chr4_GenotypeGVCFs") has been submitted
Your job 1161718 ("chr5_GenotypeGVCFs") has been submitted
Your job 1161719 ("chr6_GenotypeGVCFs") has been submitted
Your job 1161720 ("chr7_GenotypeGVCFs") has been submitted
Your job 1161721 ("chr8_GenotypeGVCFs") has been submitted
Your job 1161722 ("chr9_GenotypeGVCFs") has been submitted
Your job 1161723 ("chr10_GenotypeGVCFs") has been submitted
Your job 1161724 ("chr11_GenotypeGVCFs") has been submitted
Your job 1161725 ("chr12_GenotypeGVCFs") has been submitted
Your job 1161726 ("chr13_GenotypeGVCFs") has been submitted
Your job 1161727 ("chr14_GenotypeGVCFs") has been submitted
Your job 1161728 ("chr15_GenotypeGVCFs") has been submitted
Your job 1161729 ("chr16_GenotypeGVCFs") has been submitted
Your job 1161730 ("chr17_GenotypeGVCFs") has been

rm: cannot remove '/research/bsi/projects/PI/tertiary/Asmann_Yan_wangy3/s212975.Wickland_Immunomics/processing/TCGA/processed_data/GenotypeGVCFs_on_TCGA/nimblegen_capture_NOV19_chr1/BRCA/Blood_Normal': No such file or directory
rm: cannot remove '/research/bsi/projects/PI/tertiary/Asmann_Yan_wangy3/s212975.Wickland_Immunomics/processing/TCGA/processed_data/GenotypeGVCFs_on_TCGA/nimblegen_capture_NOV19_chr2/BRCA/Blood_Normal': No such file or directory
rm: cannot remove '/research/bsi/projects/PI/tertiary/Asmann_Yan_wangy3/s212975.Wickland_Immunomics/processing/TCGA/processed_data/GenotypeGVCFs_on_TCGA/nimblegen_capture_NOV19_chr3/BRCA/Blood_Normal': No such file or directory
rm: cannot remove '/research/bsi/projects/PI/tertiary/Asmann_Yan_wangy3/s212975.Wickland_Immunomics/processing/TCGA/processed_data/GenotypeGVCFs_on_TCGA/nimblegen_capture_NOV19_chr4/BRCA/Blood_Normal': No such file or directory
rm: cannot remove '/research/bsi/projects/PI/tertiary/Asmann_Yan_wangy3/s212975.Wickland

#### Check number of files output

In [22]:
%%bash

PROJECT=nimblegen_capture_NOV19
CANCER_NAME=BRCA

for TISSUE in Blood_Normal
do
	for i in {1..22}; do echo "chr"${i} ${TISSUE}; ls /research/bsi/projects/PI/tertiary/Asmann_Yan_wangy3/s212975.Wickland_Immunomics/processing/TCGA/processed_data/GenotypeGVCFs_on_TCGA/${PROJECT}_chr${i}/${CANCER_NAME}/${TISSUE}/*vcf |wc -l ;ls /research/bsi/projects/PI/tertiary/Asmann_Yan_wangy3/s212975.Wickland_Immunomics/processing/TCGA/processed_data/GenotypeGVCFs_on_TCGA/${PROJECT}_chr${i}/${CANCER_NAME}/${TISSUE}/*idx | wc -l;done;
done

chr1 Blood_Normal
1
1
chr2 Blood_Normal
1
1
chr3 Blood_Normal
1
1
chr4 Blood_Normal
1
1
chr5 Blood_Normal
1
1
chr6 Blood_Normal
1
1
chr7 Blood_Normal
1
1
chr8 Blood_Normal
1
1
chr9 Blood_Normal
1
1
chr10 Blood_Normal
1
1
chr11 Blood_Normal
1
1
chr12 Blood_Normal
1
1
chr13 Blood_Normal
1
1
chr14 Blood_Normal
1
1
chr15 Blood_Normal
1
1
chr16 Blood_Normal
1
1
chr17 Blood_Normal
1
1
chr18 Blood_Normal
1
1
chr19 Blood_Normal
1
1
chr20 Blood_Normal
1
1
chr21 Blood_Normal
1
1
chr22 Blood_Normal
1
1


#### Once GenotypeGVCFs output confirmed, delete combined GVCFs to conserve space

### 5. Run CatVariants on genotyped GVCFs to combine all chromosomes

##### Both together

In [23]:
%%bash

PROJECT=nimblegen_capture_NOV19
CANCER_NAME=BRCA

mkdir -p /research/bsi/projects/PI/tertiary/Asmann_Yan_wangy3/s212975.Wickland_Immunomics/processing/TCGA/processed_data/GenotypeGVCFs_on_TCGA/${PROJECT}_all_chr/${CANCER_NAME}/

for TISSUE in Blood_Normal
do 
    VCFS_LIST=''
    
    for VCF in `ls -v /research/bsi/projects/PI/tertiary/Asmann_Yan_wangy3/s212975.Wickland_Immunomics/processing/TCGA/processed_data/GenotypeGVCFs_on_TCGA/${PROJECT}_chr*/${CANCER_NAME}/${TISSUE}/BlackWhite.vcf`; 

        do
            VCFS_LIST="${VCFS_LIST} --variant ${VCF}"  ; 
         done;

	java -cp /research/bsi/projects/PI/tertiary/Asmann_Yan_wangy3/s212975.Wickland_Immunomics/tools/GenomeAnalysisTK.jar org.broadinstitute.gatk.tools.CatVariants -R /research/bsi/projects/PI/tertiary/Asmann_Yan_wangy3/s212975.Wickland_Immunomics/GRCh38.d1.vd1.fa ${VCFS_LIST} -out /research/bsi/projects/PI/tertiary/Asmann_Yan_wangy3/s212975.Wickland_Immunomics/processing/TCGA/processed_data/GenotypeGVCFs_on_TCGA/${PROJECT}_all_chr/${CANCER_NAME}/${TISSUE}_BlackWhite.vcf
done

.........10.........20..
There were no warn messages.


INFO  13:21:35,769 HelpFormatter - ------------------------------------------------------- 
INFO  13:21:35,771 HelpFormatter - Program Name: org.broadinstitute.gatk.tools.CatVariants 
INFO  13:21:35,774 HelpFormatter - Program Args: -R /research/bsi/projects/PI/tertiary/Asmann_Yan_wangy3/s212975.Wickland_Immunomics/GRCh38.d1.vd1.fa --variant /research/bsi/projects/PI/tertiary/Asmann_Yan_wangy3/s212975.Wickland_Immunomics/processing/TCGA/processed_data/GenotypeGVCFs_on_TCGA/nimblegen_capture_NOV19_chr1/BRCA/Blood_Normal/BlackWhite.vcf --variant /research/bsi/projects/PI/tertiary/Asmann_Yan_wangy3/s212975.Wickland_Immunomics/processing/TCGA/processed_data/GenotypeGVCFs_on_TCGA/nimblegen_capture_NOV19_chr2/BRCA/Blood_Normal/BlackWhite.vcf --variant /research/bsi/projects/PI/tertiary/Asmann_Yan_wangy3/s212975.Wickland_Immunomics/processing/TCGA/processed_data/GenotypeGVCFs_on_TCGA/nimblegen_capture_NOV19_chr3/BRCA/Blood_Normal/BlackWhite.vcf --variant /research/bsi/projects/PI/tertiary/Asm

### 5. Run VQSR

In [28]:
%%bash 


CANCER_NAME=BRCA
TISSUE=Blood_Normal
PROJECT=nimblegen_capture_NOV19_all_chr


#create directory for output
OUTPUT_DIR=/research/bsi/projects/PI/tertiary/Asmann_Yan_wangy3/s212975.Wickland_Immunomics/processing/TCGA/processed_data/VQSR_on_TCGA/${PROJECT}/${CANCER_NAME}/${TISSUE}
#rm -r $OUTPUT_DIR #delete if already exists
mkdir -p $OUTPUT_DIR
mkdir -p /home/mayo/m187735/s212975.Wickland_Immunomics/commands_to_qsub/TCGA/VQSR_on_TCGA/${PROJECT}/${CANCER_NAME}/${TISSUE}

INPUT_DIR=/research/bsi/projects/PI/tertiary/Asmann_Yan_wangy3/s212975.Wickland_Immunomics/processing/TCGA/processed_data/GenotypeGVCFs_on_TCGA/${PROJECT}/${CANCER_NAME}/

#set VQSR variables
REF=/research/bsi/projects/PI/tertiary/Asmann_Yan_wangy3/s212975.Wickland_Immunomics/GRCh38.d1.vd1.fa 
HAPMAP=/research/bsi/projects/PI/tertiary/Asmann_Yan_wangy3/s212975.Wickland_Immunomics/tools/VQSR/hapmap_3.3.hg38.vcf.gz
OMNI=/research/bsi/projects/PI/tertiary/Asmann_Yan_wangy3/s212975.Wickland_Immunomics/tools/VQSR/1000G_omni2.5.hg38.vcf.gz
G1000=/research/bsi/projects/PI/tertiary/Asmann_Yan_wangy3/s212975.Wickland_Immunomics/tools/VQSR/1000G_phase1.snps.high_confidence.hg38.vcf.gz
DBSNP=/research/bsi/projects/PI/tertiary/Asmann_Yan_wangy3/s212975.Wickland_Immunomics/tools/VQSR/Homo_sapiens_assembly38.dbsnp138.vcf
MILLS=/research/bsi/projects/PI/tertiary/Asmann_Yan_wangy3/s212975.Wickland_Immunomics/tools/VQSR/Mills_and_1000G_gold_standard.indels.hg38.vcf.gz


#for GROUP in {${TISSUE}_Black,${TISSUE}_White}; #NOTE: REPLACED ALL INSTANCES OF ${GROUP} WITH ${TISSUE}_BlackWhite
#	do 
		
		INPUT=${INPUT_DIR}/${TISSUE}_BlackWhite.vcf
		
		############################
		#SNPs############
		############################
		echo "java -Xmx10g -jar /research/bsi/projects/PI/tertiary/Asmann_Yan_wangy3/s212975.Wickland_Immunomics/tools/GenomeAnalysisTK.jar -T VariantRecalibrator -R ${REF} -input ${INPUT} -tranche 100.0 -tranche 99.9 -tranche 99.0 -tranche 90.0 -mode SNP -an QD -an MQ -an MQRankSum -an ReadPosRankSum -an FS -an SOR -an InbreedingCoeff -resource:hapmap,VCF,known=false,training=true,truth=true,prior=15.0 ${HAPMAP} -resource:omni,known=false,training=true,truth=true,prior=12.0 ${OMNI} -resource:1000G,known=false,training=true,truth=false,prior=10.0 ${G1000} -resource:dbsnp,known=true,training=false,truth=false,prior=2.0 ${DBSNP} -recalFile ${OUTPUT_DIR}/${TISSUE}_BlackWhite.vcf_recalibrate_SNPs -tranchesFile ${OUTPUT_DIR}/${TISSUE}_BlackWhite.vcf_tranches_SNPs -rscriptFile ${OUTPUT_DIR}/${TISSUE}_BlackWhite.vcf_VQSR_SNPs_only.plots.R --disable_auto_index_creation_and_locking_when_reading_rods --target_titv 3.2
		
		java -Xmx10g -jar /research/bsi/projects/PI/tertiary/Asmann_Yan_wangy3/s212975.Wickland_Immunomics/tools/GenomeAnalysisTK.jar -T ApplyRecalibration -R ${REF} -input ${INPUT} -o ${OUTPUT_DIR}/${TISSUE}_BlackWhite.vcf_VQSR_SNPs_only.vcf  -nt 5 -recalFile ${OUTPUT_DIR}/${TISSUE}_BlackWhite.vcf_recalibrate_SNPs -tranchesFile ${OUTPUT_DIR}/${TISSUE}_BlackWhite.vcf_tranches_SNPs --ts_filter_level 99.5 -mode SNP" > /home/mayo/m187735/s212975.Wickland_Immunomics/commands_to_qsub/TCGA/VQSR_on_TCGA/${PROJECT}/${CANCER_NAME}/${TISSUE}/${TISSUE}_BlackWhite.qsub
		
					 	
		############################
		#INDELs############
		############################
		echo "java -Xmx10g -jar /research/bsi/projects/PI/tertiary/Asmann_Yan_wangy3/s212975.Wickland_Immunomics/tools/GenomeAnalysisTK.jar -T VariantRecalibrator -R ${REF} -input ${OUTPUT_DIR}/${TISSUE}_BlackWhite.vcf_VQSR_SNPs_only.vcf -tranche 100.0 -tranche 99.9 -tranche 99.0 -tranche 90.0 -mode INDEL -an QD -an MQRankSum -an ReadPosRankSum -an FS -an SOR -an InbreedingCoeff --maxGaussians 4 -resource:mills,known=false,training=true,truth=true,prior=12.0 ${MILLS} -resource:dbsnp,known=true,training=false,truth=false,prior=2.0 ${DBSNP} -recalFile ${OUTPUT_DIR}/${TISSUE}_BlackWhite.vcf_VQSR_SNPs_recalibrate_INDELs -tranchesFile ${OUTPUT_DIR}/${TISSUE}_BlackWhite.vcf_VQSR_SNPs_tranches_INDELs --disable_auto_index_creation_and_locking_when_reading_rods		
		
		
		java -Xmx10g -jar /research/bsi/projects/PI/tertiary/Asmann_Yan_wangy3/s212975.Wickland_Immunomics/tools/GenomeAnalysisTK.jar -T ApplyRecalibration -R ${REF} -input ${OUTPUT_DIR}/${TISSUE}_BlackWhite.vcf_VQSR_SNPs_only.vcf -o ${OUTPUT_DIR}/${TISSUE}_BlackWhite_final_VQSR.vcf -recalFile ${OUTPUT_DIR}/${TISSUE}_BlackWhite.vcf_VQSR_SNPs_recalibrate_INDELs -tranchesFile ${OUTPUT_DIR}/${TISSUE}_BlackWhite.vcf_VQSR_SNPs_tranches_INDELs --ts_filter_level 99 -mode INDEL" >> /home/mayo/m187735/s212975.Wickland_Immunomics/commands_to_qsub/TCGA/VQSR_on_TCGA/${PROJECT}/${CANCER_NAME}/${TISSUE}/${TISSUE}_BlackWhite.qsub
		
		
		#submit job
		rm /research/bsi/projects/PI/tertiary/Asmann_Yan_wangy3/s212975.Wickland_Immunomics/commands_to_qsub/TCGA/VQSR_on_TCGA/${PROJECT}/${CANCER_NAME}/${TISSUE}/${TISSUE}_BlackWhite.err 2> /dev/null
		rm /research/bsi/projects/PI/tertiary/Asmann_Yan_wangy3/s212975.Wickland_Immunomics/commands_to_qsub/TCGA/VQSR_on_TCGA/${PROJECT}/${CANCER_NAME}/${TISSUE}/${TISSUE}_BlackWhite.out 2> /dev/null
		
		qsub -l h_vmem=20G -e /home/mayo/m187735/s212975.Wickland_Immunomics/commands_to_qsub/TCGA/VQSR_on_TCGA/${PROJECT}/${CANCER_NAME}/${TISSUE}/${TISSUE}_BlackWhite.err -o /home/mayo/m187735/s212975.Wickland_Immunomics/commands_to_qsub/TCGA/VQSR_on_TCGA/${PROJECT}/${CANCER_NAME}/${TISSUE}/${TISSUE}_BlackWhite.out -q 4-day -pe threaded 6 -N VQSR_${PROJECT} /home/mayo/m187735/s212975.Wickland_Immunomics/commands_to_qsub/TCGA/VQSR_on_TCGA/${PROJECT}/${CANCER_NAME}/${TISSUE}/${TISSUE}_BlackWhite.qsub;





Your job 1161946 ("VQSR_nimblegen_capture_NOV19_all_chr") has been submitted


## DATA PROCESSING AND ANALYSIS

###  1. Split VQSR file by chromosome to speed up analysis
• This will take ~20 minutes  
• Note the need to rename sample IDs in VCF header. This is to facilitate matching of sample IDs in VCF (current format: H_LS-3C-AALK-10A-01D-A41F-09) to sample IDs in the master table file (current format: TCGA-3C-AALK) that's based on TCGA metdata. Matching IDs is necessary so that the low-coverage, medium-coverage and high-coverage samples from the master table can be extracted separately from the plink MAF file that's generated from the VCFs.  
• Also note that the bam file had an irregular name for one of the samples that does not resemble its true TCGA ID, hence the manual substitution.

In [29]:
%%bash

PROJECT=nimblegen_capture_NOV19
CANCER_NAME=BRCA

for TISSUE in Blood_Normal;
    do
        VQSR_OUTPUT_FILE=/research/bsi/projects/PI/tertiary/Asmann_Yan_wangy3/s212975.Wickland_Immunomics/processing/TCGA/processed_data/VQSR_on_TCGA/${PROJECT}_all_chr/${CANCER_NAME}/${TISSUE}/${TISSUE}_BlackWhite_final_VQSR.vcf;

               for i in {1..22};
                    do
                        mkdir -p `dirname ${VQSR_OUTPUT_FILE}`/chr${i}

                        cat <(grep -v ^## ${VQSR_OUTPUT_FILE} |  head -n1 | sed 's/H_LS/TCGA/g' | sed 's/Pooled_DNA-2011-03/TCGA-AQ-A04L/' | sed 's/\t/\n/g' | cut -d '-' -f1-3 | sed 's/\n/\t/g' | tr '\n' '\t') <(echo) <(awk -v CHR=$i '$1=="chr"CHR' ${VQSR_OUTPUT_FILE}) > `dirname ${VQSR_OUTPUT_FILE}`/chr${i}/`basename ${VQSR_OUTPUT_FILE} .vcf`_chr.vcf &
                    done;
          
    done


### 2. Subset the PASS variants and annotate using Annovar (including population allele frequencies)

In [39]:
%%bash 

for i in {1..22}; do 
    
    CANCER_NAME=BRCA
    TISSUE=Blood_Normal
    PROJECT=nimblegen_capture_NOV19_all_chr/${CANCER_NAME}/${TISSUE}/chr${i}

    #create directory for output
    OUTPUT_DIR=/research/bsi/projects/PI/tertiary/Asmann_Yan_wangy3/s212975.Wickland_Immunomics/processing/TCGA/processed_data/annotate_PASS_TCGA/${PROJECT}/
    #rm -r $OUTPUT_DIR #delete if already exists
    mkdir -p $OUTPUT_DIR
    mkdir -p /home/mayo/m187735/s212975.Wickland_Immunomics/commands_to_qsub/TCGA/annotate_PASS_TCGA/${PROJECT}

    INPUT_DIR=/research/bsi/projects/PI/tertiary/Asmann_Yan_wangy3/s212975.Wickland_Immunomics/processing/TCGA/processed_data/VQSR_on_TCGA/${PROJECT}/


    INPUT=${INPUT_DIR}/${TISSUE}_BlackWhite_final_VQSR_chr.vcf

    INPUT_PASS=${OUTPUT_DIR}/`basename ${INPUT} .vcf`_PASS.vcf

    echo "awk '/#/ || /PASS/' ${INPUT}> ${INPUT_PASS}     		

    cd /research/bsi/projects/PI/tertiary/Asmann_Yan_wangy3/s212975.Wickland_Immunomics/tools/annovar/

    perl /research/bsi/projects/PI/tertiary/Asmann_Yan_wangy3/s212975.Wickland_Immunomics/tools/annovar/table_annovar.pl ${INPUT_PASS} humandb -buildver hg38 --out ${INPUT_PASS} -remove -protocol refGene,dbnsfp30a,EUR.sites.2015_08,AFR.sites.2015_08 -operation gx,f,f,f -nastring . -vcfinput " > /home/mayo/m187735/s212975.Wickland_Immunomics/commands_to_qsub/TCGA/annotate_PASS_TCGA/${PROJECT}/BlackWhite.qsub;




    rm /research/bsi/projects/PI/tertiary/Asmann_Yan_wangy3/s212975.Wickland_Immunomics/commands_to_qsub/TCGA/annotate_PASS_TCGA/${PROJECT}/*err 2> /dev/null
    rm /research/bsi/projects/PI/tertiary/Asmann_Yan_wangy3/s212975.Wickland_Immunomics/commands_to_qsub/TCGA/annotate_PASS_TCGA/${PROJECT}/*out 2> /dev/null;

    for file in /research/bsi/projects/PI/tertiary/Asmann_Yan_wangy3/s212975.Wickland_Immunomics/commands_to_qsub/TCGA/annotate_PASS_TCGA/${PROJECT}/*qsub; do qsub -l h_vmem=20G -e /home/mayo/m187735/s212975.Wickland_Immunomics/commands_to_qsub/TCGA/annotate_PASS_TCGA/${PROJECT}/`basename ${file} .qsub`.err -o /home/mayo/m187735/s212975.Wickland_Immunomics/commands_to_qsub/TCGA/annotate_PASS_TCGA/${PROJECT}/`basename ${file} .qsub`.out -q 1-day -pe threaded 4 -N `basename ${PROJECT}`_annotate $file;done;
done

Your job 1162508 ("chr1_annotate") has been submitted
Your job 1162509 ("chr2_annotate") has been submitted
Your job 1162510 ("chr3_annotate") has been submitted
Your job 1162511 ("chr4_annotate") has been submitted
Your job 1162512 ("chr5_annotate") has been submitted
Your job 1162513 ("chr6_annotate") has been submitted
Your job 1162514 ("chr7_annotate") has been submitted
Your job 1162515 ("chr8_annotate") has been submitted
Your job 1162516 ("chr9_annotate") has been submitted
Your job 1162517 ("chr10_annotate") has been submitted
Your job 1162518 ("chr11_annotate") has been submitted
Your job 1162519 ("chr12_annotate") has been submitted
Your job 1162520 ("chr13_annotate") has been submitted
Your job 1162521 ("chr14_annotate") has been submitted
Your job 1162522 ("chr15_annotate") has been submitted
Your job 1162523 ("chr16_annotate") has been submitted
Your job 1162524 ("chr17_annotate") has been submitted
Your job 1162525 ("chr18_annotate") has been submitted
Your job 1162526 ("

#### If black and white genotyped together, must now create separated VCFs for each race

In [40]:
%%bash

PROJECT=nimblegen_capture_NOV19
for TISSUE in Blood_Normal;
   do
    for RACE in Black White;
        do 
            for i in {1..22};
                do 
                    cat /research/bsi/projects/PI/tertiary/Asmann_Yan_wangy3/s212975.Wickland_Immunomics/processing/TCGA/bams_lists_hg38/${PROJECT}_chr${i}/BRCA_gvcf_list_${TISSUE}_${RACE}.txt | cut -d '/' -f14 | sed 's/\.g\.vcf//g' | cut -d '-' -f1-3> /research/bsi/projects/PI/tertiary/Asmann_Yan_wangy3/s212975.Wickland_Immunomics/processing/TCGA/processed_data/annotate_PASS_TCGA/${PROJECT}_all_chr/BRCA/${TISSUE}/chr${i}/${RACE}_sample_list.txt


                    /research/bsi/projects/PI/tertiary/Asmann_Yan_wangy3/s212975.Wickland_Immunomics/tools/vcftools_0.1.13/bin/vcftools --vcf /research/bsi/projects/PI/tertiary/Asmann_Yan_wangy3/s212975.Wickland_Immunomics/processing/TCGA/processed_data/annotate_PASS_TCGA/${PROJECT}_all_chr/BRCA/${TISSUE}/chr${i}/${TISSUE}_BlackWhite_final_VQSR_chr_PASS.vcf.hg38_multianno.vcf --keep /research/bsi/projects/PI/tertiary/Asmann_Yan_wangy3/s212975.Wickland_Immunomics/processing/TCGA/processed_data/annotate_PASS_TCGA/${PROJECT}_all_chr/BRCA/${TISSUE}/chr${i}/${RACE}_sample_list.txt --recode --recode-INFO-all --out /research/bsi/projects/PI/tertiary/Asmann_Yan_wangy3/s212975.Wickland_Immunomics/processing/TCGA/processed_data/annotate_PASS_TCGA/${PROJECT}_all_chr/BRCA/${TISSUE}/chr${i}/${TISSUE}_BlackWhite_${RACE}_final_VQSR_chr_PASS.vcf.hg38_multianno.vcf
                done;
        done;
    done;



VCFtools - v0.1.13
(C) Adam Auton and Anthony Marcketta 2009

Parameters as interpreted:
	--vcf /research/bsi/projects/PI/tertiary/Asmann_Yan_wangy3/s212975.Wickland_Immunomics/processing/TCGA/processed_data/annotate_PASS_TCGA/nimblegen_capture_NOV19_all_chr/BRCA/Blood_Normal/chr1/Blood_Normal_BlackWhite_final_VQSR_chr_PASS.vcf.hg38_multianno.vcf
	--keep /research/bsi/projects/PI/tertiary/Asmann_Yan_wangy3/s212975.Wickland_Immunomics/processing/TCGA/processed_data/annotate_PASS_TCGA/nimblegen_capture_NOV19_all_chr/BRCA/Blood_Normal/chr1/Black_sample_list.txt
	--recode-INFO-all
	--out /research/bsi/projects/PI/tertiary/Asmann_Yan_wangy3/s212975.Wickland_Immunomics/processing/TCGA/processed_data/annotate_PASS_TCGA/nimblegen_capture_NOV19_all_chr/BRCA/Blood_Normal/chr1/Blood_Normal_BlackWhite_Black_final_VQSR_chr_PASS.vcf.hg38_multianno.vcf
	--recode

Keeping individuals in 'keep' list
After filtering, kept 161 out of 742 Individuals
Outputting VCF file...
After filtering, kept 39253 out

### 3. Downsample Whites so that have equivalent number of both racial groups (optional)

In [5]:
%%R

PREFIX <- 'nimblegen_capture_NOV19'

for (CHROM in seq(1,22)){
  
  HEADER <- system(paste0("grep -v ^## /research/bsi/projects/PI/tertiary/Asmann_Yan_wangy3/s212975.Wickland_Immunomics/processing/TCGA/processed_data/annotate_PASS_TCGA/",PREFIX,"_all_chr/BRCA/Blood_Normal/chr",CHROM,"/Blood_Normal_BlackWhite_White_final_VQSR_chr_PASS.vcf.hg38_multianno.vcf.recode.vcf | head -n1"),intern = TRUE)
  HEADER <- strsplit(HEADER,split="\t")[[1]]
  
  VCF <- read.table(paste0("/research/bsi/projects/PI/tertiary/Asmann_Yan_wangy3/s212975.Wickland_Immunomics/processing/TCGA/processed_data/annotate_PASS_TCGA/",PREFIX,"_all_chr/BRCA/Blood_Normal/chr",CHROM,"/Blood_Normal_BlackWhite_White_final_VQSR_chr_PASS.vcf.hg38_multianno.vcf.recode.vcf"),sep='\t',header=FALSE)
  
  names(VCF) <- HEADER
  
  
  NUMBER_TO_SELECT = 161
  
  set.seed(10) #to ensure same ones selected for each chromosome VCF!
  
  RANDOM_VCFS <- sample(names(VCF)[grepl('TCGA',names(VCF))],size = NUMBER_TO_SELECT)
  
  
  VCF <- cbind(VCF[names(VCF)[!grepl('TCGA',names(VCF))]],VCF[,RANDOM_VCFS])
  
  write.table(VCF,paste0("/research/bsi/projects/PI/tertiary/Asmann_Yan_wangy3/s212975.Wickland_Immunomics/processing/TCGA/processed_data/annotate_PASS_TCGA/",PREFIX,"_all_chr/BRCA/Blood_Normal/chr",CHROM,"/Blood_Normal_BlackWhite_White_DOWNSAMPLED_final_VQSR_chr_PASS.vcf.hg38_multianno.vcf.recode.vcf"),sep='\t',row.names=FALSE,quote=FALSE)
}

### 4. Retrieve the population-level (1000 Genomes) MAFs and calculate per-locus depths (DP) (BY COVERAGE LEVEL) from annotated VCFs
- Keep all BIALLELIC loci, regardless of VQSLOD and genotype

#### Script: [extract_annotated_population_MAFs_cohort_DPs_and_MAFs_by_coverage_level.R](./Scripts/extract_annotated_population_MAFs_cohort_DPs_and_MAFs_by_coverage_level.R)

In [6]:
%%bash

PROJECT=nimblegen_capture_NOV19
CANCER_NAME=BRCA


for TISSUE in Blood_Normal
    do
    PREFIX=${PROJECT}_all_chr/${CANCER_NAME}/${TISSUE}/chr
    for i in {1..22};
        do
            for RACE in BlackWhite_White_DOWNSAMPLED; 

                do
           echo "/research/bsi/projects/PI/tertiary/Asmann_Yan_wangy3/s212975.Wickland_Immunomics/tools/R-3.6.2/bin/Rscript --vanilla /home/mayo/m187735/s212975.Wickland_Immunomics/GitHub/Racial_Disparities_in_Exome_Read_Depth/Scripts/extract_annotated_population_MAFs_cohort_DPs_and_MAFs_by_coverage_level.R ${PREFIX}${i} ${RACE} ${TISSUE}" | qsub -l h_vmem=1G -e /home/mayo/m187735/s212975.Wickland_Immunomics/commands_to_qsub/TCGA/annotate_PASS_TCGA/${PREFIX}${i}/${TISSUE}_${RACE}_population_MAF_cohort_DPs_and_MAFs_by_coverage_level.err -o /home/mayo/m187735/s212975.Wickland_Immunomics/commands_to_qsub/TCGA/annotate_PASS_TCGA/${PREFIX}${i}/${TISSUE}_${RACE}_population_MAF_cohort_DPs_and_MAFs_by_coverage_level.out -N  extr_chr${i}_${RACE} -q 1-day,4-day,lg-mem -pe threaded 1
                done;
        done;
    done

Your job 1433035 ("extr_chr1_BlackWhite_White_DOWNSAMPLED") has been submitted
Your job 1433036 ("extr_chr2_BlackWhite_White_DOWNSAMPLED") has been submitted
Your job 1433037 ("extr_chr3_BlackWhite_White_DOWNSAMPLED") has been submitted
Your job 1433038 ("extr_chr4_BlackWhite_White_DOWNSAMPLED") has been submitted
Your job 1433039 ("extr_chr5_BlackWhite_White_DOWNSAMPLED") has been submitted
Your job 1433040 ("extr_chr6_BlackWhite_White_DOWNSAMPLED") has been submitted
Your job 1433041 ("extr_chr7_BlackWhite_White_DOWNSAMPLED") has been submitted
Your job 1433042 ("extr_chr8_BlackWhite_White_DOWNSAMPLED") has been submitted
Your job 1433043 ("extr_chr9_BlackWhite_White_DOWNSAMPLED") has been submitted
Your job 1433044 ("extr_chr10_BlackWhite_White_DOWNSAMPLED") has been submitted
Your job 1433045 ("extr_chr11_BlackWhite_White_DOWNSAMPLED") has been submitted
Your job 1433046 ("extr_chr12_BlackWhite_White_DOWNSAMPLED") has been submitted
Your job 1433047 ("extr_chr13_BlackWhite_White_DO

### 4. Extract *all* 1000 Genomes positions from common capture region; want to compare Black vs. White MAF in that region


In [1]:
%%bash

 awk '{print "chr"$1,$2,$3,$4,$5}' OFS="\t" /research/bsi/projects/PI/tertiary/Asmann_Yan_wangy3/s212975.Wickland_Immunomics/processing/TCGA/1000Genomes/1000Genomes_all_chr_all_variants.vcf > /research/bsi/projects/PI/tertiary/Asmann_Yan_wangy3/s212975.Wickland_Immunomics/processing/TCGA/1000Genomes/1000Genomes_all_chr_all_variants_nochr.vcf


In [6]:
%%bash

mkdir /research/bsi/projects/PI/tertiary/Asmann_Yan_wangy3/s212975.Wickland_Immunomics/processing/TCGA/processed_data/capture_kit/1000Genomes_within_common_capture_region


for i in {1..22}; do /research/bsi/projects/PI/tertiary/Asmann_Yan_wangy3/s212975.Wickland_Immunomics/tools/vcftools_0.1.13/bin/vcftools --vcf /research/bsi/projects/PI/tertiary/Asmann_Yan_wangy3/s212975.Wickland_Immunomics/processing/TCGA/1000Genomes/1000Genomes_all_chr_all_variants_nochr.vcf --bed /research/bsi/projects/PI/tertiary/Asmann_Yan_wangy3/s212975.Wickland_Immunomics/processing/TCGA/beds/intersection_three_nimblegen_kits_hg38_DW_chr${i}.bed  --recode --out  /research/bsi/projects/PI/tertiary/Asmann_Yan_wangy3/s212975.Wickland_Immunomics/processing/TCGA/processed_data/capture_kit/1000Genomes_within_common_capture_region/chr${i} & done

mkdir: cannot create directory '/research/bsi/projects/PI/tertiary/Asmann_Yan_wangy3/s212975.Wickland_Immunomics/processing/TCGA/processed_data/capture_kit/1000Genomes_within_common_capture_region': File exists

VCFtools - v0.1.13
(C) Adam Auton and Anthony Marcketta 2009

Parameters as interpreted:
	--vcf /research/bsi/projects/PI/tertiary/Asmann_Yan_wangy3/s212975.Wickland_Immunomics/processing/TCGA/1000Genomes/1000Genomes_all_chr_all_variants_nochr.vcf
	--out /research/bsi/projects/PI/tertiary/Asmann_Yan_wangy3/s212975.Wickland_Immunomics/processing/TCGA/processed_data/capture_kit/1000Genomes_within_common_capture_region/chr1
	--recode
	--bed /research/bsi/projects/PI/tertiary/Asmann_Yan_wangy3/s212975.Wickland_Immunomics/processing/TCGA/beds/intersection_three_nimblegen_kits_hg38_DW_chr1.bed


VCFtools - v0.1.13
(C) Adam Auton and Anthony Marcketta 2009

Parameters as interpreted:
	--vcf /research/bsi/projects/PI/tertiary/Asmann_Yan_wangy3/s212975.Wickland_Immunomics/processing/TCG

In [8]:
%%bash 

for i in {1..22}; do cat /research/bsi/projects/PI/tertiary/Asmann_Yan_wangy3/s212975.Wickland_Immunomics/processing/TCGA/processed_data/capture_kit/1000Genomes_within_common_capture_region/chr${i}.recode.vcf >> /research/bsi/projects/PI/tertiary/Asmann_Yan_wangy3/s212975.Wickland_Immunomics/processing/TCGA/processed_data/capture_kit/1000Genomes_within_common_capture_region/1000Genomes_within_common_capture_region_all_chr.vcf;done